In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Occasions Annotations

## Bin of 3-5 `guest_count`
Let's look at a few tables:
- `korea_toronto_hocky_20190410_20190424.csv`
- `toronto_silvester_20181223_20190105.csv`
- `toronto_valentine_20190210_20190216.csv`

to get an idea of the different occasions.

Our initial set of **occasions** for the 3-5 bin is:
1. `Business Date`
2. `Dinner Date`
3. `Social Gathering`
4. `Family Event`
5. `Party`
6. `Drinking`
7. `After Work`

Something to think about:
- `Lunch`
- `Dinner`

----------

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
hockey_path = '../data/korea_toronto_hocky_20190410_20190424.csv'
silvester_path = '../data/toronto_silvester_20181223_20190105.csv'
valentine_path = '../data/toronto_valentine_20190210_20190216.csv'

In [5]:
hockey = pd.read_csv(hockey_path)
silvester = pd.read_csv(silvester_path)
valentine = pd.read_csv(valentine_path)

Fix `datetime` features:

In [6]:
def fix_times(df):
    df['order_time'] = pd.to_datetime(df['order_time'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_time_closed'] = pd.to_datetime(df['order_time_closed'], format="%Y-%m-%d %H:%M:%S.%f")
    df['order_item_time'] = pd.to_datetime(df['order_item_time'], format="%Y-%m-%d %H:%M:%S.%f")

In [7]:
fix_times(hockey)
fix_times(valentine)
fix_times(silvester)

Create `dwell_time` feature:

In [8]:
def set_dwell_time(df):
    df['dwell_time'] = df['order_time_closed'] - df['order_time']

In [9]:
set_dwell_time(hockey)
set_dwell_time(valentine)
set_dwell_time(silvester)

Drop the `bar_id` feature, it's useless here.

In [10]:
features = ['order_id', 'order_time', 'order_time_closed', 'dwell_time', 'order_item_id', 'order_item_time',
            'title', 'category_id', 'beer_brand_id', 'beer_serving_type_id', 'beer_volume',
            'item_qty', 'sales_before_tax', 'sales_inc_tax', 'guest_count']

Reorder columns:

In [11]:
hockey = hockey[features]
valentine = valentine[features]
silvester = silvester[features]

In [12]:
hockey.head()

,order_id,order_time,order_time_closed,dwell_time,order_item_id,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
0,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,01:14:52,52000810601,2019-04-16 12:13:26,$Sub.Swt.Pot.Fry-,2.0,0,0,0.0,1.0,2.97,3.425081,1
1,520008106,2019-04-16 12:01:00,2019-04-16 13:15:52,01:14:52,52000810600,2019-04-16 12:13:26,FINGERS.SAUCED,2.0,0,0,0.0,1.0,16.87,19.454919,1
2,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,01:10:55,52000810502,2019-04-16 12:18:25,TRAD.GUAC,2.0,0,0,0.0,1.0,8.99,10.368399,1
3,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,01:10:55,52000810501,2019-04-16 12:13:26,Add.Veg--,2.0,0,0,0.0,1.0,0.00,0.000000,1
4,520008105,2019-04-16 12:01:00,2019-04-16 13:11:55,01:10:55,52000810500,2019-04-16 12:13:26,VEGGIE.BURGER,2.0,0,0,0.0,1.0,17.62,20.321601,1


Create table for each order, we're going to pick (randomly) 10 tables out of each dataframe.

In [13]:
def create_tables(df):
    tables = df.drop_duplicates(subset='order_id', keep='first')
    tables.set_index('order_id', inplace=True)
    tables = tables[['guest_count']]

    return tables

In [14]:
hockey_tables = create_tables(hockey)
valentine_tables = create_tables(valentine)
silvester_tables = create_tables(silvester)

Keep only relevant features:

In [15]:
hockey.drop(['order_item_id'], axis=1, inplace=True)
valentine.drop(['order_item_id'], axis=1, inplace=True)
silvester.drop(['order_item_id'], axis=1, inplace=True)

In [16]:
hockey.set_index('order_id', inplace=True)
valentine.set_index('order_id', inplace=True)
silvester.set_index('order_id', inplace=True)

-----

## The Annotations

Pick 10 tables out of each dataframe.

In [17]:
COUNT = 10

In [18]:
import random

In [19]:
def get_tables_3to5(tables):
    return tables[(tables['guest_count'] >= 3) & (tables['guest_count'] <= 5)]

### Hockey

Pick 10 tables of 3 to 5 guests of **Hockey**.

In [20]:
hockey_tables_3to5 = get_tables_3to5(hockey_tables)
print("We have {} tables of 3 to 5 guests".format(len(hockey_tables_3to5)))

We have 13640 tables of 3 to 5 guests


In [21]:
# picked_hockey_tables = random.sample(list(hockey_tables_3to5.index), COUNT)

In [22]:
picked_hockey_tables = [
    520803608,
    523294848,
    514583822,
    524421347,
    521095118,
    522574215,
    522779155,
    515363895,
    519501788,
    514529424
]

In [23]:
print("Got {} tables".format(len(picked_hockey_tables)))

Got 10 tables


#### 1. Table 520803608

In [24]:
hockey[hockey.index == 520803608].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Steak & Eggs,2.0,0,0,0.000,1.0,15.99,18.069068,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Luau Chicken,2.0,0,0,0.000,1.0,15.99,18.069068,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Freddy Tuna Sal,2.0,0,0,0.000,1.0,12.99,14.678999,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:29:13,Daily Soup,2.0,0,0,0.000,1.0,4.99,5.638815,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:31:40,8oz Pinot Grigio,6.0,0,0,0.000,1.0,10.25,11.582736,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:31:40,Von Bugle TB,1.0,8462,3,0.473,1.0,4.00,4.520092,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 13:31:40,Von Bugle TB,1.0,8462,3,0.473,1.0,4.00,4.520092,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 14:38:02,Coffee,4.0,0,0,0.000,1.0,2.69,3.039762,3
520803608,2019-04-17 13:28:24,2019-04-17 14:44:51,01:16:27,2019-04-17 14:38:02,Espresso DBL,4.0,0,0,0.000,1.0,2.75,3.107563,3


April 17th (Wednesday), 13:28 - 14:44, 3 guests.

Meal Steps Description:
1. Start
    - 3 Dishes (+1 Small Soup):
        - Steak & Eggs
        - Luau Chicken
        - Freddy Tuna Sal
        - Daily Soup
    - 3 Drinks:
        - Pinot Grigio Wine
        - Von Bugle Beer Can x 2
2. One Hour Later:
    - 3 Coffee Drinks:
        - Coffee
        - Espresso Double
        - Cafe Latte

`Lunch`

#### 2. Table 523294848

In [25]:
hockey[hockey.index == 523294848].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,FAJ.CHICKEN,2.0,0,0,0.000,1.0,21.81,24.646138,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,CHED&BEER.SOUP,2.0,0,0,0.000,1.0,7.98,9.017707,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,HEINEKEN.Pt,1.0,3,2,0.562,1.0,9.33,10.543259,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,POP.,4.0,0,0,0.000,1.0,3.99,4.508853,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 11:57:19,FAJ.FOR.1,2.0,0,0,0.000,1.0,0.00,0.000000,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,K.COOKIE,2.0,0,0,0.000,1.0,0.00,0.000000,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,K.BUTTER.PASTA,2.0,0,0,0.000,1.0,7.81,8.825600,4
523294848,2019-04-20 11:57:00,2019-04-20 12:40:43,00:43:43,2019-04-20 12:14:52,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,4


April 20th (Saturday), 11:57 - 12:40, 4 guests.

Meal Steps Description:
1. Start
    - 2 Dishes:
        - Flavored Chicken (for 1)
        - Cheddar & Beer Soup
    - 2 Drinks:
        - Heineken PINT
        - Pop
2. 15 Minutes Later:
    - Dishes:
        - Kids Pasta Butter
        - Kids Burger
        - Italian Burger
        - Broccoli Bites

`Family Event`

#### 3. Table 514583822

In [26]:
hockey[hockey.index == 514583822].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:46:54,BUDWEISER.Jg,1.0,71,2,1.538,1.0,21.72,24.543706,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:47:27,QUESADILLAS,2.0,0,0,0.000,1.0,14.97,16.916173,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:47:51,WINGS,2.0,0,0,0.000,1.0,0.00,0.000000,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 19:47:51,WINGS.15PC,2.0,0,0,0.000,1.0,23.65,26.724615,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:15:38,BUDWEISER.Jg,1.0,71,2,1.538,1.0,21.72,24.543706,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:23:22,QUESADILLAS,2.0,0,0,0.000,1.0,14.97,16.916173,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:23:29,CHK.PARMIGIANA,2.0,0,0,0.000,1.0,20.87,23.583202,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:35:20,w.hot sauce,2.0,0,0,0.000,1.0,0.00,0.000000,4
514583822,2019-04-13 19:46:00,2019-04-13 21:33:29,01:47:29,2019-04-13 20:35:20,n/c.sauce,2.0,0,0,0.000,1.0,0.00,0.000000,4


April 13th (Saturday), 19:46 - 21:33, 4 guests.

Meal Steps Description:
1. Start
    - 2 Dishes:
        - Quesadillas
        - Wings
    - 1 Drink:
        - Budweiser
2. Half an Hour Later:
    - 1 Drink:
        - Budweiser
3. 8 Minutes Later:
    - 2 Dishes:
        - Quesadillas
        - Chicken Parmigiana
4. Half an Hour Later:
    - 1 Drink:
        - Pop
5. 15 Minutes Later:
    - 1 Drink:
        - Budweiser

`Social Gathering`

#### 4. Table 524421347

In [27]:
hockey[hockey.index == 524421347].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS.10PC,2.0,0,0,0.0,1.0,16.99,19.198043,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,KIDPOP,4.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,Add.Poutine^,2.0,0,0,0.0,1.0,5.94,6.711970,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,JACKS.VEG.NACHOS,2.0,0,0,0.0,1.0,21.26,24.022978,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS,2.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,Add.Fries^,2.0,0,0,0.0,1.0,2.97,3.355985,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS,2.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,KIDPOP,4.0,0,0,0.0,1.0,0.00,0.000000,3
524421347,2019-04-21 17:31:00,2019-04-21 18:39:40,01:08:40,2019-04-21 17:31:04,WINGS.10PC,2.0,0,0,0.0,1.0,16.99,19.198043,3


April 21th (Sunday), 17:31 - 18:39, 3 guests.

Meal Steps Description:
1. Start:
    - Dishes:
        - Wings + Poutine
        - Veggie Nachos
        - Wings + Fries
    - 1 Drink:
        - Mint Soda
2. Half an Hour Later:
    - 2 Dishes:
        - Poutine
        - Chicken Fingers

`Family Event`

#### 5. Table 521095118

In [28]:
hockey[hockey.index == 521095118].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,JAMESON,3.0,0,0,0.000,1.0,6.00,6.780042,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,JAMESON,3.0,0,0,0.000,1.0,6.00,6.780042,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,PATRON SLVR,3.0,0,0,0.000,1.0,13.07,14.769192,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,LG COORS LIGHT,1.0,280,2,0.591,1.0,7.25,8.192551,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,JAMESON,3.0,0,0,0.000,1.0,6.00,6.780042,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,LRG BANQUET,1.0,1120,2,0.591,1.0,7.75,8.757555,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,LRG RICKARDS,1.0,191,2,0.591,1.0,7.75,8.757555,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,PATRON SLVR,3.0,0,0,0.000,1.0,13.07,14.769192,4
521095118,2019-04-17 21:20:00,2019-04-18 00:39:38,03:19:38,2019-04-17 21:18:00,PATRON SLVR,3.0,0,0,0.000,1.0,13.07,14.769192,4


April 17th (Wednesday), 21:20 - 00:40, 4 guests.

Meal Steps Description:
1. Start:
    - 6 Beers:
        - Coors Light x 4
        - Large Banquet
        - Large Rickards
    - 12 Alcoholic Drinks:
        - Jameson x 3
        - Patron Silver Tequila x 4
        - Captain Morgan
        - Crown Royal x 4
    - Wine:
        - Red Sangria
2. 15 Minutes Later:
    - 1 Beer:
        - Bud Light
    - 5 Alcohol Drinks:
        - Jameson x 4
        - Lamb's Rum
3. 20 Minutes Later:
    - 3 Beers:
        - Corona
        - Coors Light
        - Bud Light
    -  5 Alcoholic Drinks:
        - Crown Royal x 3
        - Wiser
        - Cosmopolitan
4. 20 Minutes Later:
    - 3 Beers:
        - Banquet x 3
5. 35 Minutes Later:
    - 1 Beer:
        - Stella
6. 35 Minutes Later:
    - 2 Beers:
        - Coors Light x 2
    - 1 Alcoholic Drink:
        - Jack Daniels
7. 35 Minutes Later:
    - 2 Beers:
        - Coors Light x 2
    - 1 Alcoholic Drink:
        - Jack Daniels
8. 20 Minutes Later:
    - 2 Beers:
        - Coors Light x 2
    - 1 Alcoholic Drink:
        - Jack Daniels

`Drinking`

#### 6. Table 522574215

In [29]:
hockey[hockey.index == 522574215].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,K- Chic Fingers,2.0,0,0,0.0,1.0,8.99,10.158057,4
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,ED 6oz SteakFrit,2.0,0,0,0.0,1.0,11.99,13.547843,4
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,K- Ling Pasta,2.0,0,0,0.0,1.0,8.99,10.158057,4
522574215,2019-04-19 17:18:05,2019-04-19 18:17:19,00:59:14,2019-04-19 17:19:13,Super Berry Kale,2.0,0,0,0.0,1.0,18.98,21.446043,4


April 19th (Friday), 17:18 - 18:17, 4 guests.

Meal Steps Description:
1. Start:
    - 4 Dishes:
        - Kids Chicken Fingers
        - Kids Pasta
        - Steak & Fries
        - Kale Salad

`Family Event`

#### 7. Table 522779155

In [30]:
hockey[hockey.index == 522779155].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 00:47:00,HEINEKEN,1.0,3,1,0.33,1.0,7.20,8.136950,4
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 00:47:00,WISER SPL BL,5.0,0,0,0.00,1.0,5.79,6.543464,4
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 01:00:00,WISER SPL BL,5.0,0,0,0.00,1.0,5.79,6.543464,4
522779155,2019-04-20 00:47:00,2019-04-20 01:17:53,00:30:53,2019-04-20 01:00:00,GREY GOOSE,3.0,0,0,0.00,1.0,8.50,9.606122,4


April 20th (Saturday), 00:47 - 01:17, 4 guests.

1. Start:
    - 2 Side Dishes (WISER SPL BL)
2. 13 Minutes Later:
    - 2 Drinks:
        - Heineken
        - Grey Goose

`Drinking` (we want to believe the *WISER SPL BL* is really a drink rather than a side dish)

#### 8. Table 515363895

In [31]:
hockey[hockey.index == 515363895].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,PB Big Breakfast,2.0,0,0,0.0,1.0,14.99,16.937782,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378517,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,PB French Toast,2.0,0,0,0.0,1.0,10.99,12.418027,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,PB French Toast,2.0,0,0,0.0,1.0,12.98,14.666605,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,Coffee,4.0,0,0,0.0,1.0,2.69,3.039535,3
515363895,2019-04-14 10:52:19,2019-04-14 11:53:30,01:01:11,2019-04-14 10:53:24,Coffee,4.0,0,0,0.0,1.0,2.69,3.039535,3


April 14th (Sunday), 10:52 - 11:53, 3 guests.

1. Start:
    - 3 Dishes:
        - Breakfast
        - French Toast x 2
    - 3 Drinks:
        - Fount pop
        - Coffee x 2

`Breakfast` (/ `Brunch`)

#### 9. Table 519501788

In [32]:
hockey[hockey.index == 519501788].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:29:42,COORS.LIGHT.Pt,1.0,280,2,0.473,1.0,7.62,8.610963,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:29:42,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:29:42,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,TRIPLE.BAC.CHSBRG,2.0,0,0,0.000,1.0,20.97,23.697099,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,DBL.CHSBURG,2.0,0,0,0.000,1.0,16.68,18.849195,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,DBL.BAC.CHSBRG,2.0,0,0,0.000,1.0,17.87,20.193952,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:41:57,K.BURGER,2.0,0,0,0.000,1.0,8.98,10.147828,4
519501788,2019-04-15 18:29:00,2019-04-15 19:22:30,00:53:30,2019-04-15 18:55:28,COORS.LIGHT.Pt,1.0,280,2,0.473,1.0,7.62,8.610963,4


April 15th (Monday), 18:29 - 19:22, 4 guests.

1. Start:
    - 2 Beers:
        - Coors Light PINT x 2
    - 4 Dishes:
        - Cheese Burger x 3
        - Kids Burger

`Family Event`

#### 10. Table 514529424

In [33]:
hockey[hockey.index == 514529424].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 17:55:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 17:55:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:05:00,POTATO SKINS,2.0,0,0,0.0,1.0,5.95,6.722456,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:05:00,CALAMARI,2.0,0,0,0.0,1.0,5.95,6.722456,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:05:00,SOFT TACO (BEEF),2.0,0,0,0.0,1.0,5.95,6.722456,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:24:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3
514529424,2019-04-13 17:55:00,2019-04-13 19:07:57,01:12:57,2019-04-13 18:34:00,CAESAR VODKA DBL,3.0,0,0,0.0,1.0,10.50,11.863158,3


April 13th (Saturday), 17:55 - 19:07, 3 guests.

1. Start:
    - 3 Dishes:
        - Potato Skins
        - Calamari
        - Soft Taco
    - 4 Drinks:
        - Caesar Vodka x 4

`Social Gathering`

### Valentine

Pick 10 tables of 3 to 5 guests of **Valentine**.

In [34]:
valentine_tables_3to5 = get_tables_3to5(valentine_tables)
print("We have {} tables of 3 to 5 guests".format(len(valentine_tables_3to5)))

We have 4859 tables of 3 to 5 guests


In [35]:
# picked_valentine_tables = random.sample(list(valentine_tables_3to5.index), COUNT)

In [36]:
picked_valentine_tables = [
    447441456,
    435005397,
    435011399,
    447381509,
    446744761,
    447079489,
    434609975,
    448041560,
    447074978,
    434603220
]

In [37]:
print("Got {} tables".format(len(picked_valentine_tables)))

Got 10 tables


#### 1. Table 447441456

In [38]:
valentine[valentine.index == 447441456].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:06:00,BANQUET.XL,1.0,1120,2,0.591,1.0,10.01,11.309856,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:13:00,BRUSCHETTA.,2.0,0,0,0.000,1.0,13.31,15.038380,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,$Sub.Swt.Pot.Fry-,2.0,0,0,0.000,1.0,2.97,3.355671,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,K.WINGS,2.0,0,0,0.000,1.0,8.98,10.146104,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,FAJ.BOWL.CHK,2.0,0,0,0.000,1.0,19.99,22.585816,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:15:00,K.PASTA CHS,2.0,0,0,0.000,1.0,7.81,8.824173,3
447441456,2019-02-14 17:06:00,2019-02-14 18:27:42,01:21:42,2019-02-14 17:43:00,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,3


February 14th (Thursday), 17:06 - 18:27, 3 guests.

1. Start:
    - 1 Beer:
        - Banquet
2. 10 Minutes Later:
    - Dishes:
        - Bruschetta
        - Chicken
        - Kids Pasta Cheese
        - Kids Wings

`Family Event`

#### 2. Table 435005397

In [39]:
valentine[valentine.index == 435005397].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:05:00,LG GUINNESS,1.0,26,2,0.591,1.0,8.75,9.887636,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:05:00,LRG BANQUET,1.0,1120,2,0.591,1.0,7.75,8.757621,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:17:00,SPRING ROLLS,2.0,0,0,0.000,1.0,12.00,13.560187,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:20:00,CHICKEN BITES,2.0,0,0,0.000,1.0,14.00,15.820218,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:24:00,ICE TEA,4.0,0,0,0.000,1.0,3.25,3.672551,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:34:00,LG GUINNESS,1.0,26,2,0.591,1.0,8.75,9.887636,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:34:00,LRG BANQUET,1.0,1120,2,0.591,1.0,7.75,8.757621,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:40:00,NACHOS SUPREME,2.0,0,0,0.000,1.0,16.00,18.080249,3
435005397,2019-02-11 12:05:00,2019-02-11 13:56:33,01:51:33,2019-02-11 12:40:00,NEW YORK 16 OZ,2.0,0,0,0.000,1.0,32.00,36.160498,3


February 11th (Monday), 12:05 - 13:56, 3 guests.

1. Start:
    - 2 Beers:
        - Large Guinness
        - Large Banquet
2. 15 Minutes Later:
    - 2 Dishes:
        - Spring Rolls
        - Chicken Bites
    - 1 Drink:
        - Ice Tea
3. 10 Minutes Later:
    - 2 Beers:
        - Large Guinness
        - Large Banquet
    - 3 Dishes:
        - Nachos
        - Burger x 2
4. 40 Minutes Later:
    - 2 Beers:
        - Large Guinness
        - Large Banquet

`Social Gathering`

#### 3. Table 435011399

In [40]:
valentine[valentine.index == 435011399].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,Create Omelette,2.0,0,0,0.0,1.0,11.69,13.379721,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,*No Bev,5.0,0,0,0.0,1.0,0.00,0.000000,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,Freddy Tuna Sal,2.0,0,0,0.0,1.0,11.69,13.379721,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:06:33,Freddy Tuna Sal,2.0,0,0,0.0,1.0,11.69,13.379721,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:56:14,Coffee,4.0,0,0,0.0,1.0,2.43,2.781242,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:56:14,Coffee,4.0,0,0,0.0,1.0,2.42,2.769797,3
435011399,2019-02-11 13:05:24,2019-02-11 14:01:59,00:56:35,2019-02-11 13:56:14,Coffee Decaf,4.0,0,0,0.0,1.0,2.42,2.769797,3


February 11th (Monday), 13:05 - 14:02, 3 guests.

1. Start:
    - 3 Dishes:
        - Omelette
        - Freddy Tuna Salad
2. 50 Minutes Later:
    - 3 Drinks:
        - Coffee x 3

`Lunch`

#### 4. Table 447381509

In [41]:
valentine[valentine.index == 447381509].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:56:59,8oz Lucky SHRZ,6.0,0,0,-0.0,-1.0,0.00,0.000000,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:56:59,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378595,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Crispy Calamari,2.0,0,0,0.0,1.0,13.99,15.808210,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Maple Nut Salmon,2.0,0,0,0.0,1.0,24.99,28.237825,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,10oz Steak Frite,2.0,0,0,0.0,1.0,29.99,33.887650,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Pete's Deli,2.0,0,0,0.0,1.0,16.99,19.198105,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:57:44,Pete's Deli,2.0,0,0,-0.0,-1.0,0.00,0.000000,3
447381509,2019-02-14 13:56:36,2019-02-14 14:38:53,00:42:17,2019-02-14 13:59:57,8oz Deakin SHZ,6.0,0,0,0.0,1.0,11.00,12.429615,3


**???**

February 14th (Thursday), 13:56 - 14:38, 3 guests.

1. Start:
    - 3 (/4 ?) Dishes:
        - Crispy Calamari
        - Maple Nut Salmon
        - Steak Frite
        - Deli (?)
    - 1 Wine:
        - Shiraz

`Lunch`

#### 5. Table 446744761

In [42]:
valentine[valentine.index == 446744761].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:15:00,BADOIT.750ML,4.0,0,0,0.0,1.0,6.56,7.412722,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:25:00,BRUSCHETTA.,2.0,0,0,0.0,1.0,13.31,15.040142,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:25:00,LOBSTER.CRB.DIP,2.0,0,0,0.0,1.0,14.97,16.915922,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:25:00,CALAMARI,2.0,0,0,0.0,1.0,14.35,16.215329,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:35:00,BOMBAY.CHICKEN,2.0,0,0,0.0,1.0,19.97,22.565863,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:35:00,TUNA.SALAD+,2.0,0,0,0.0,1.0,19.87,22.452864,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:35:00,TUNA.BOWL,2.0,0,0,0.0,1.0,20.77,23.469853,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:54:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508653,3
446744761,2019-02-12 11:15:00,2019-02-12 12:57:53,01:42:53,2019-02-12 11:54:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508653,3


February 12th (Tuesday), 11:15 - 12:57, 3 guests.

1. Start:
    - 1 Drink:
        - Badot Water
2. 10 Minutes Later:
    - 3 Dishes:
        - Bruschetta
        - Lobster
        - Calamari
3. 10 Minutes Later:
    - 3 Dishes:
        - Chicken
        - Tuna Salad
        - Tuna
4. 20 Minutes Later:
    - 2 Drinks:
        - Pop x 2

`Lunch`

#### 6. Table 447079489

In [43]:
valentine[valentine.index == 447079489].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Maple Nut Salmon,2.0,0,0,0.0,1.0,19.99,22.588469,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Grill Cheeseburg,2.0,0,0,0.0,1.0,15.99,18.068515,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,*Water,4.0,0,0,0.0,1.0,0.00,0.000000,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Chicken Parmesan,2.0,0,0,0.0,1.0,18.99,21.458481,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 18:25:48,Fish & Chips,2.0,0,0,0.0,1.0,13.99,15.808538,4
447079489,2019-02-13 18:25:29,2019-02-13 19:14:24,00:48:55,2019-02-13 19:11:01,Fount Pop,4.0,0,0,0.0,3.0,8.97,10.135996,4


February 13th (Wednesday), 18:25 - 19:14, 4 guests.

1. Start:
    - 4 Dishes:
        - Maple Nut Salmon
        - Grill Cheeseburger
        - Chicken Parmesan
        - Fish & Chips
    - 1 Drink:
        - 1 Water
2. 45 Minutes Later:
    - 1 Drink:
        - 1 Pop

`Dinner`

#### 7. Table 434609975

In [44]:
valentine[valentine.index == 434609975].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:32,8oz Drifting CAB,6.0,0,0,0.0,1.0,12.00,13.559475,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:32,8oz Brancott SB,6.0,0,0,0.0,1.0,12.00,13.559475,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:39,S-Pork Potstickr,2.0,0,0,0.0,1.0,5.99,6.768438,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:13:48,K- Chic Fingers,2.0,0,0,0.0,1.0,11.98,13.536876,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:20:30,Salmon Teriyaki,2.0,0,0,0.0,1.0,19.99,22.587825,3
434609975,2019-02-10 13:12:45,2019-02-10 14:17:33,01:04:48,2019-02-10 13:20:30,Tuna Poke Bowl,2.0,0,0,0.0,1.0,17.99,20.327912,3


February 10th (Sunday), 13:12 - 14:17, 3 guests.

1. Start:
    - 4 Dishes:
        - Potstickr
        - Kids Chicken Fingers
        - Salmon
        - Tuna
    - 2 Wines:
        - Drifting Cabernet Sauvignon
        - Brancott

`Family Event`

#### 8. Table 448041560

In [45]:
valentine[valentine.index == 448041560].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,EXTRA Egg,2.0,0,0,0.000,1.0,1.99,2.248707,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Canadian,1.0,183,1,0.341,1.0,5.50,6.215018,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Freddy Tuna Sal,2.0,0,0,0.000,1.0,12.99,14.678744,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Maple Nut Salmon,2.0,0,0,0.000,1.0,24.99,28.238784,3
448041560,2019-02-15 19:39:25,2019-02-15 20:44:38,01:05:13,2019-02-15 19:40:42,Meatloaf,2.0,0,0,0.000,1.0,13.99,15.808747,3


February 15th (Friday), 19:39 - 20:44, 3 guests.

1. Start:
    - 3 Dishes:
        - Tuna Salad
        - Maple Nut Salmon
        - Meatloaf
    - 1 Drink:
        - Canadian Beer

`Dinner`

#### 9. Table 447074978

In [46]:
valentine[valentine.index == 447074978].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,Add.Plain.Mash--,2.0,0,0,0.000,1.0,0.00,0.000000,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,QUESA.STACK,2.0,0,0,0.000,1.0,18.59,21.007192,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,Add.Plain.Mash--,2.0,0,0,0.000,1.0,0.00,0.000000,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063547,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,CALAMARI,2.0,0,0,0.000,1.0,14.35,16.215880,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063547,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,CAULI.BITES+,2.0,0,0,0.000,1.0,13.38,15.119754,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063547,4
447074978,2019-02-13 17:57:00,2019-02-13 19:25:12,01:28:12,2019-02-13 17:57:00,SUB.PARM.POUT,2.0,0,0,0.000,1.0,5.97,6.746258,4


February 13th (Wednesday), 17:57 - 19:25, 4 guests.

1. Start:
    - 4 Dishes + 3 Sharables:
        - Quesadilla
        - Chicken Fingers x 3
        - Calamari
        - Cauliflower Bites
        - Poutine
2. 30 Minutes Later:
    - 3 Soft Drinks:
        - Water x 2
        - Pop
    - 1 Beer:
        - Lagunita PINT
3. 30 Minutes Later:
    - 1 Dish:
        - Cookie & Milk

`After Work`

#### 10. Table 434603220

In [47]:
valentine[valentine.index == 434603220].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:26:17,Coffee,4.0,0,0,0.0,1.0,2.469298,2.790400,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:26:17,Coffee,4.0,0,0,0.0,1.0,2.469298,2.790400,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,PB Big Breakfast,2.0,0,0,0.0,1.0,13.760140,15.549476,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,Greek LG,2.0,0,0,0.0,1.0,20.626440,23.308653,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,The B.B.B.L.T,2.0,0,0,0.0,1.0,13.760140,15.549476,3
434603220,2019-02-10 12:26:12,2019-02-10 13:47:54,01:21:42,2019-02-10 12:44:42,Fount Pop,4.0,0,0,0.0,1.0,2.744684,3.101597,3


February 10th (Sunday), 12:26 - 13:47, 3 guests.

1. Start:
    - 2 Drinks:
        - Coffee x 2
2. 20 Minutes Later:
    - 3 Dishes:
        - Breakfast
        - Greek
        - ???
    - 1 Drink:
        - Pop

`Lunch`

### Silvester

Pick 10 tables of 3 to 5 guests of **Silvester**.

In [48]:
silvester_tables_3to5 = get_tables_3to5(silvester_tables)
print("We have {} tables of 3 to 5 guests".format(len(silvester_tables_3to5)))

We have 14636 tables of 3 to 5 guests


In [49]:
# picked_silvester_tables = random.sample(list(silvester_tables_3to5.index), COUNT)

In [50]:
picked_silvester_tables = [
    362939931,
    361304992,
    363233153,
    360790021,
    362924064,
    361776660,
    361815160,
    360000762,
    360402335,
    361369624
]

In [51]:
print("Got {} tables".format(len(picked_silvester_tables)))

Got 10 tables


#### 1. Table 362939931

In [52]:
silvester[silvester.index == 362939931].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
362939931,2019-01-02 18:23:00,2019-01-02 19:59:38,01:36:38,2019-01-02 19:20:00,LG SMITHWICKS,1.0,720,2,0.591,1.0,8.75,9.886598,3
362939931,2019-01-02 18:23:00,2019-01-02 19:59:38,01:36:38,2019-01-02 19:20:00,BUFFALO CHIC WRAP,2.0,0,0,0.000,1.0,15.50,17.513402,3


January 2nd (Wednesday), 18:23 - 19:59, 3 guests.

1. Start:
    - 1 Dish:
        - 1 Buffalo Chicken
    - 1 Drink:
        - Smithwicks Beer

`After Work`

#### 2. Table 361304992

In [53]:
silvester[silvester.index == 361304992].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:25:00,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:25:00,ONTARIO.CRAFT.XL,1.0,2482,2,0.591,1.0,10.30,11.636786,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:25:00,KIDBEV,4.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,n/c.sauce,2.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,garlic.aioli,2.0,0,0,0.000,1.0,0.00,0.000000,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,$$Truffle.Fries$$,2.0,0,0,0.000,1.0,2.97,3.355462,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,K.PIZZA,2.0,0,0,0.000,1.0,8.98,10.145470,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,ITALIAN.BURGER**,2.0,0,0,0.000,1.0,17.62,19.906813,3
361304992,2018-12-29 15:25:00,2018-12-29 16:13:18,00:48:18,2018-12-29 15:29:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3


December 29th (Saturday), 15:25 - 16:13, 3 guests.

1. Start:
    - 4 Dishes:
        - Truffle Fries
        - Italian Burger
        - Kids Pizza x 2
    - Soft Drinks:
        - Kids Pop x 2
    - 1 Beer:
        - Ontario Craft

`Family Event`

#### 3. Table 363233153

In [54]:
silvester[silvester.index == 363233153].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:41:00,ALABAMA SLAMMER,3.0,0,0,0.0,1.0,6.75,7.591907,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,PENNE SUNDRIED,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,MAC 'N CHEESE,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,MAC 'N CHEESE,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,GRILLED CHEESE 3.0,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,VEGGIE BURGER,2.0,0,0,0.0,1.0,5.95,6.692125,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,GRILLED CHEESE 3.0,2.0,0,0,0.0,1.0,7.90,8.885343,5
363233153,2019-01-03 12:41:00,2019-01-03 13:34:41,00:53:41,2019-01-03 12:42:00,VEGGIE BURGER,2.0,0,0,0.0,1.0,5.95,6.692125,5


January 3rd (Thursday), 12:41 - 13:34, 5 guests.

1. Start:
    - Dishes:
        - Pasta
        - Mac & Cheese x 2
        - Grilled Cheese x 2
        - Veggie Burger x 2
    - 1 Cocktail:
        - Alabama Slammer

`Lunch`

#### 4. Table 360790021

In [55]:
silvester[silvester.index == 360790021].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:21:41,Killer Shake,4.0,0,0,0.0,1.0,4.99,5.638489,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,PB Jumbo Club,2.0,0,0,0.0,1.0,14.48,16.361787,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,SIDE Pickles,2.0,0,0,-0.0,-1.0,0.00,0.000000,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,Pesto Chicken,2.0,0,0,0.0,1.0,18.99,21.457896,3
360790021,2018-12-28 14:21:35,2018-12-28 14:57:45,00:36:10,2018-12-28 14:26:14,Grilled Cheese,2.0,0,0,0.0,1.0,13.48,15.231829,3


December 28th (Friday), 14:21 - 14:57, 3 guests.

1. Start:
    - 3 Dishes:
        - Jumbo Club
        - Pesto Chicken
        - Grilled Cheese
    - 1 Soft Drink:
        - Killer Shake

`Lunch`

#### 5. Table 362924064

In [56]:
silvester[silvester.index == 362924064].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:29:00,POP.,4.0,0,0,0.0,1.0,3.99,4.509349,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:29:00,POP.,4.0,0,0,0.0,1.0,3.99,4.509349,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:35:00,CHICKEN.FINGERS,2.0,0,0,0.0,1.0,15.99,18.071302,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:35:00,TUNA.BOWL,2.0,0,0,0.0,1.0,0.00,0.000000,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:35:00,Side.Caesar,2.0,0,0,0.0,1.0,0.00,0.000000,3
362924064,2019-01-02 17:29:00,2019-01-02 18:45:58,01:16:58,2019-01-02 17:56:00,SESAME CHICKEN,2.0,0,0,0.0,1.0,0.00,0.000000,3


January 2nd (Wednesday), 17:29 - 18:45, 3 guests.

1. Start:
    - 2 Soft Drinks:
        - Pop x 2
2. 5 Minutes Later:
    - 1 Dish:
        - Chicken Fingers

`After Work`

#### 6. Table 361776660

In [57]:
silvester[silvester.index == 361776660].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,Side Caesar.,2.0,0,0,0.000,1.0,0.00,0.000000,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,ASIAGO.BOWTIE,2.0,0,0,0.000,1.0,19.99,22.589221,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,WINGS.5PC,2.0,0,0,0.000,1.0,10.47,11.831373,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,FRIED.CHK.SAND,2.0,0,0,0.000,1.0,17.09,19.312145,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,BREWSKY.RT.BEER,4.0,0,0,0.000,1.0,4.45,5.028616,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,ADD CAESAR,2.0,0,0,0.000,1.0,5.25,5.932637,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,WINGS..,2.0,0,0,0.000,1.0,0.00,0.000000,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,CHICKEN.FINGERS,2.0,0,0,0.000,1.0,15.99,18.069117,4
361776660,2018-12-30 19:03:00,2018-12-30 20:27:17,01:24:17,2018-12-30 19:03:00,POP.,4.0,0,0,0.000,1.0,3.99,4.508804,4


December 30th (Sunday), 19:03 - 20:27, 4 guests.

1. Start:
    - Dishes:
        - Asiago & BowTie
        - Wingos
        - Fried Chicken
        - Chicken Fingers
    - Beers:
        - Brewsky Beer
        - Mill Street
    - Soft Drinks:
        - Pop
2. 20 Minutes Later:
    - 1 Cocktai
3. 40 Minutes Later:
    - 2 Desserts:
        - Ice Cream
        - Brownie Cookie

`Dinner`

#### 7. Table 361815160

In [58]:
silvester[silvester.index == 361815160].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,PEROGIES,2.0,0,0,0.000,1.0,5.95,6.680803,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,VEGGIE BURGER,2.0,0,0,0.000,1.0,7.90,8.870309,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,CHICKEN SANDWICH,2.0,0,0,0.000,1.0,6.95,7.803627,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,VIRGIN RASP MOJITO,4.0,0,0,0.000,1.0,3.75,4.210590,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:14:58,OLD FASHIONED,3.0,0,0,0.000,1.0,12.25,13.754594,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:40:42,BUDWEISER TAP,1.0,71,2,0.414,1.0,6.00,6.736944,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 20:40:42,PEROGIES,2.0,0,0,0.000,1.0,5.95,6.680803,5
361815160,2018-12-30 20:14:00,2018-12-30 21:09:42,00:55:42,2018-12-30 21:00:28,COKE,4.0,0,0,0.000,1.0,2.95,3.312331,5


December 30th (Sunday), 20:14 - 21:09, 5 guests.

1. Start:
    - 3 Dishes:
        - Perogies
        - Veggie Burger
        - Chicken Sandwich
    - 1 Soft Drink:
        - Virgin Rasp Mojito
    - 1 Cocktail:
        - Old Fashioned
2. 25 Minutes Later:
    - 1 Beer:
        - Budweiser
    - 1 Dish:
        - Perogies
3. 20 Minutes Later:
    - 1 Soft Drink:
        - Coke

`Dinner`

#### 8. Table 360000762

In [59]:
silvester[silvester.index == 360000762].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Salmon,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Starter,2.0,0,0,0.000,1.0,0.0,0.00,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Salmon,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Sirloin,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,ED Turkey Bologn,2.0,0,0,0.000,1.0,18.0,20.34,4
360000762,2018-12-26 16:30:50,2018-12-26 17:32:00,01:01:10,2018-12-26 16:31:30,****COURSE 2****,2.0,0,0,0.000,1.0,0.0,0.00,4


December 26th (Wednesday), 16:30 - 17:32, 4 guests.

1. Start:
    - 4 Dishes:
        - Salmon x 2
        - Sirolin Steak
        - Turkey Bologn
    - 1 Beer:
        - Goose Island
    - 3 Wines:
        - Black River
        - Chateau Char x 2

`Dinner`

#### 9. Table 360402335

In [60]:
silvester[silvester.index == 360402335].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:19:13,5ozDriftng CHARD,6.0,0,0,0.000,1.0,7.50,8.475289,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:19:13,8oz Pinot Grigio,6.0,0,0,0.000,1.0,10.00,11.300386,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:19:13,16oz Shock Top,1.0,267,2,0.473,1.0,6.75,7.627760,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:28:53,Salmon Teriyaki,2.0,0,0,0.000,1.0,18.99,21.459432,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:28:53,Salmon Teriyaki,2.0,0,0,0.000,1.0,18.99,21.459432,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 18:28:53,PB House Burger,2.0,0,0,0.000,1.0,14.99,16.939278,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 19:21:55,Froyo Strawberry,2.0,0,0,0.000,1.0,2.99,3.378815,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 19:46:43,Coffee Decaf,4.0,0,0,0.000,1.0,2.69,3.039804,3
360402335,2018-12-27 18:18:52,2018-12-27 19:49:49,01:30:57,2018-12-27 19:46:43,Coffee,4.0,0,0,0.000,1.0,2.69,3.039804,3


December 27th (Thursday), 18:18 - 19:49, 3 guests.

1. Start:
    - 2 Wines:
        - Drifting Chardonnay
        - Pinot grigio
    - 1 Beer:
        - Shock Top
2. 10 Minutes Later:
    - 3 Dishes:
        - Salmon x 2
        - Burger
3. 50 Minutes Later:
    - 1 Dish:
        - Froyo Strawberry
4. 25 Minutes Later:
    - 2 Soft Drinks:
        - Coffee x 2

`Dinner`

#### 10. Table 361369624

In [61]:
silvester[silvester.index == 361369624].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:31:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,ASIAGO.BOWTIE,2.0,0,0,0.0,1.0,19.99,22.590156,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,PAD THAI,2.0,0,0,0.0,1.0,0.00,0.000000,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,ADD PAN BREAD,2.0,0,0,0.0,1.0,5.25,5.932882,5
361369624,2018-12-29 19:31:00,2018-12-29 20:30:15,00:59:15,2018-12-29 19:42:00,DELUXE.BURGER,2.0,0,0,0.0,1.0,17.73,20.036191,5


December 29th (Saturday), 19:31 - 20:30, 5 guests.

1. Start:
    - 5 Water
2. 10 Minutes Later:
    - 5 Dishes:
        - Asiago BowTie
        - Burger x 2
        - Pad Thai Chicken
        - Fajita Steak
    - 1 Beer:
        - Brewsky Beer

`Family Event`

-----

### Conclusion

The relevant occasions are:
1. ~~`Business Date`~~
2. ~~`Dinner Date`~~
3. `Social Gathering`
4. `Family Event`
5. ~~`Party`~~
6. `Drinking`
7. `After Work`
8. `Lunch`
9. `Dinner`

Or:
1. `Social Gathering`  
    No time-specific event.  
    Contains beers / shareable dishes / many meal steps.
2. `Family Event`  
    Contains **K.** / **K-** or any other kids-related dish (pancakes, milkshake etc.).  
3. `Drinking`  
    A lot of alcoholic drinks compared to number of dishes.
4. `After Work`  
    After work hours, less shareable dishes compared to a social-gathering event, contains more main dishes.
5. `Lunch`  
    Lunch period, mostly contains main dishes rather than desserts / alcohol.  
    Number of main dishes usually equals to the number of guests.
6. `Dinner`  
    Dinner period, mostly contains main dishes rather than desserts / alcohol.  
    Number of main dishes usually equals to the number of guests.

-----

## Conclusion Update Sep. 15th

After further examinations, we've decided on the following occasions:
1. `Family Event`:  
    Any table that contains kids-related dishes; whether it contains *K.* / *K-* / *KID* in the title or a specified set of dishes / drinks (pancakes, milkshake etc.)  
2. `Drinking`:  
    Contains mostly alcoholic drinks (no main dishes at all)  
3. ~ Meal-Based event:  
    Contains mostly main dishes.  
    Split to the following occasions, based on the ToD (time of day):  
    a. `Breakfast`  
    b. ~~`Brunch`~~  
    c. `Lunch`  
    d. `Dinner`  
4. `Social Gathering`; contains shareable dishes w/ drinks, sits somewhere in the middle between `Lunch` and `Drinking`:  
    This one is further split to the following occasions, based on the ToD (time of day):  
    a. `Social Gathering`  
    b. `After Work`  


### In summary
we have the following occasions:
1. `Family Event`
2. `Drinking`
3. `Breakfast`
4. `Lunch`
5. `Dinner`
6. `Social Gathering`
7. `After Work`

--------------

## CONTINUE

We'll be using the 30 annotated tables above for DEV set.  
Let's add 60 additional tables; 30 for DEV, 30 for test, total of 20 additional for each data file.

In [62]:
left_hockey_tables = [index for index in list(hockey_tables_3to5.index) if index not in picked_hockey_tables]
left_valentine_tables = [index for index in list(valentine_tables_3to5.index) if index not in picked_valentine_tables]
left_silvester_tables = [index for index in list(silvester_tables_3to5.index) if index not in picked_silvester_tables]

In [63]:
COUNT = 20

In [64]:
# additional_hockey_tables = random.sample(left_hockey_tables, COUNT)

In [65]:
additional_hockey_tables = [
    513486993, 522542767, 523317944, 520779953, 514329317, 524196111, 525550553, 519525238, 513506080, 520130914,
    523514188, 523328731, 521804862, 524442000, 514472097, 521770033, 520119333, 525397783, 511353667, 522420188
]

In [66]:
# additional_valentine_tables = random.sample(left_valentine_tables, COUNT)

In [67]:
additional_valentine_values = [
    446798723, 448169651, 434708231, 447647574, 448003813, 447986271, 447020385, 447091085, 435006071, 434752383,
    447523773, 435067253, 434617535, 447986377, 447877104, 447558662, 448008007, 447497220, 435171863, 447019409
]

In [68]:
# additional_silvester_tables = random.sample(left_silvester_tables, COUNT)

In [69]:
additional_silvester_tables = [
    363177655, 359978168, 359644089, 362094000, 361772528, 359978861, 360862630, 362626040, 359970875, 363694039,
    361723648, 360802830, 360778622, 360543623, 363655255, 360057302, 361881265, 360336485, 359420117, 361326979
]

------------

### Hockey

#### 1. Table 513486993

In [70]:
hockey[hockey.index == 513486993].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:17:41,Lychee Lemonade,3.0,0,0,0.0,1.0,7.99,9.028756,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:17:41,Bellini,3.0,0,0,0.0,1.0,7.99,9.028756,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:17:41,Moscow Mule,3.0,0,0,0.0,1.0,7.99,9.028756,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:27:42,PB House Burger,2.0,0,0,0.0,1.0,14.99,16.938804,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:27:42,Double Monty,4.0,0,0,0.0,1.0,15.99,18.068811,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:27:42,Spanish Paella,2.0,0,0,0.0,1.0,17.99,20.328825,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 18:27:42,Pete's Deli,2.0,0,0,0.0,1.0,16.99,19.198818,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 19:04:39,Cake Choc Fudge,2.0,0,0,0.0,1.0,7.99,9.028756,4
513486993,2019-04-12 18:16:59,2019-04-12 19:12:20,00:55:21,2019-04-12 19:04:50,Coffee,4.0,0,0,0.0,1.0,2.69,3.039719,4


`Dinner`

#### 2. Table 522542767

In [71]:
hockey[hockey.index == 522542767].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 15:44:02,Green Monster,4.0,0,0,0.0,1.0,5.49,6.203498,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 15:44:02,Green Monster,4.0,0,0,0.0,1.0,5.49,6.203498,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 15:57:29,ED Fish & Chips,2.0,0,0,0.0,1.0,11.99,13.548260,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 15:57:29,Pesto Chicken,2.0,0,0,0.0,1.0,18.99,21.458003,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 15:57:29,K- Grill Cheese,2.0,0,0,0.0,1.0,7.99,9.028407,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 15:57:29,K- Spag&Meatball,2.0,0,0,0.0,1.0,9.99,11.288333,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 16:14:46,SIDE Toast,2.0,0,0,0.0,1.0,1.49,1.683645,4
522542767,2019-04-19 15:43:57,2019-04-19 16:58:01,01:14:04,2019-04-19 16:46:03,SIDE Toast,2.0,0,0,-0.0,-1.0,-1.49,-1.683645,4


`Family Event`

#### 3. Table 523317944

In [72]:
hockey[hockey.index == 523317944].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:35:55,KIDPOP,4.0,0,0,0.0,1.0,0.00,0.000000,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:35:55,HOT.TEA*,4.0,0,0,0.0,1.0,3.99,4.508888,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:35:55,POP.,4.0,0,0,0.0,1.0,3.99,4.508888,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:41:32,ADD PAN W/CHS,2.0,0,0,0.0,1.0,6.39,7.221001,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:54:37,VEGGIE.BURGER,2.0,0,0,0.0,1.0,17.62,19.911430,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:54:37,S/O.Bf.Gravy,2.0,0,0,0.0,1.0,1.49,1.683770,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:54:37,KIDS.,2.0,0,0,0.0,1.0,0.00,0.000000,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:54:37,K.WINGS,2.0,0,0,0.0,1.0,8.98,10.147823,3
523317944,2019-04-20 12:35:00,2019-04-20 13:39:32,01:04:32,2019-04-20 12:54:37,Add.Veg--,2.0,0,0,0.0,1.0,0.00,0.000000,3


`Family Event`

#### 4. Table 520779953

In [73]:
hockey[hockey.index == 520779953].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
520779953,2019-04-17 11:41:00,2019-04-17 12:39:54,00:58:54,2019-04-17 11:41:50,FINGERS & FRIES,2.0,0,0,0.000,1.0,5.95,6.674047,5
520779953,2019-04-17 11:41:00,2019-04-17 12:39:54,00:58:54,2019-04-17 11:41:50,SOUR KEY,3.0,0,0,0.000,1.0,6.75,7.571397,5
520779953,2019-04-17 11:41:00,2019-04-17 12:39:54,00:58:54,2019-04-17 11:41:50,CORONA,1.0,128,1,0.355,1.0,7.50,8.412664,5
520779953,2019-04-17 11:41:00,2019-04-17 12:39:54,00:58:54,2019-04-17 11:41:50,CHICK CSR WRAP,2.0,0,0,0.000,1.0,5.95,6.674047,5
520779953,2019-04-17 11:41:00,2019-04-17 12:39:54,00:58:54,2019-04-17 11:41:50,MAC 'N CHEESE,2.0,0,0,0.000,1.0,8.20,9.197846,5


`Social Gathering`

#### 5. Table 514329317

In [74]:
hockey[hockey.index == 514329317].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,P.B Deli Sand,2.0,0,0,0.0,1.0,12.99,14.678783,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378719,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,Freddy Tuna Sal,2.0,0,0,0.0,1.0,12.99,14.678783,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378719,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,Grilled Cheese,2.0,0,0,0.0,1.0,11.99,13.548776,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,P.B Deli Sand,2.0,0,0,0.0,1.0,12.99,14.678783,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378719,4
514329317,2019-04-13 12:55:17,2019-04-13 14:40:09,01:44:52,2019-04-13 12:56:20,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378719,4


`Lunch`

#### 6. Table 524196111

In [75]:
hockey[hockey.index == 524196111].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 11:39:33,Classic Burger,2.0,0,0,0.0,1.0,14.48,16.361441,4
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 11:39:33,P.B Deli Sand,2.0,0,0,0.0,1.0,12.99,14.677839,4
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 11:39:33,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378502,4
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 11:39:33,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378502,4
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 11:39:33,Chicken Spinach,2.0,0,0,0.0,1.0,14.99,16.937707,4
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 11:39:33,Beet & Chicken,2.0,0,0,0.0,1.0,16.99,19.197574,4
524196111,2019-04-21 11:38:43,2019-04-21 12:44:49,01:06:06,2019-04-21 12:22:35,*SIDE Onion Ring,2.0,0,0,0.0,1.0,3.99,4.508436,4


`Lunch`

#### 7. Table 525550553

In [76]:
hockey[hockey.index == 525550553].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 19:26:53,V-Daiq Strwbrry,4.0,0,0,0.0,1.0,4.49,5.073647,3
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 19:26:53,V-Daiq Strwbrry,4.0,0,0,0.0,1.0,4.49,5.073647,3
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 19:26:53,Lobster Jumbo,2.0,0,0,0.0,1.0,21.99,24.848443,3
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 19:26:53,Mushroom Swiss,2.0,0,0,0.0,1.0,14.99,16.938525,3
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 19:26:53,Grill Cheeseburg,2.0,0,0,0.0,1.0,15.99,18.068513,3
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 19:46:46,Coconut Shrimp,3.0,0,0,0.0,1.0,11.99,13.548560,3
525550553,2019-04-23 19:26:29,2019-04-23 20:18:33,00:52:04,2019-04-23 20:17:17,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378665,3


`Dinner`

#### 8. Table 519525238

In [77]:
hockey[hockey.index == 519525238].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
519525238,2019-04-15 20:45:00,2019-04-15 21:30:25,00:45:25,2019-04-15 20:45:34,AMSTERDM.SEAS.Pt,1.0,1562,2,0.473,1.0,8.72,9.853657,3
519525238,2019-04-15 20:45:00,2019-04-15 21:30:25,00:45:25,2019-04-15 20:45:34,CORONA BTL,1.0,128,1,0.355,1.0,7.41,8.373348,3
519525238,2019-04-15 20:45:00,2019-04-15 21:30:25,00:45:25,2019-04-15 20:46:34,POP.,4.0,0,0,0.000,1.0,3.99,4.508726,3
519525238,2019-04-15 20:45:00,2019-04-15 21:30:25,00:45:25,2019-04-15 20:50:32,DBL.BAC.CHSBRG,2.0,0,0,0.000,1.0,17.87,20.193216,3
519525238,2019-04-15 20:45:00,2019-04-15 21:30:25,00:45:25,2019-04-15 20:50:32,FISH'N'CHIPS,2.0,0,0,0.000,1.0,18.29,20.667819,3
519525238,2019-04-15 20:45:00,2019-04-15 21:30:25,00:45:25,2019-04-15 20:50:32,CHK.PARMIGIANA,2.0,0,0,0.000,1.0,20.87,23.583235,3


`Dinner`

#### 9. Table 513506080

In [78]:
hockey[hockey.index == 513506080].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:36:56,Matzo Ball Soup,2.0,0,0,0.0,1.0,4.99,5.638463,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,K- Grill Cheese,2.0,0,0,0.0,1.0,7.99,9.028320,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,Half BBQ Chicken,2.0,0,0,0.0,1.0,14.99,16.937987,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,K- Shake Van,4.0,0,0,0.0,1.0,2.99,3.378558,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,Beet & Chicken,2.0,0,0,0.0,1.0,16.99,19.197892,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,K- Burger,2.0,0,0,0.0,1.0,10.48,11.841902,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,K- Grill Cheese,2.0,0,0,0.0,1.0,7.99,9.028320,5
513506080,2019-04-12 19:36:50,2019-04-12 20:26:43,00:49:53,2019-04-12 19:46:21,K- Float Van,4.0,0,0,0.0,1.0,2.99,3.378558,5


`Family Event`

#### 10. Table 520130914

In [79]:
hockey[hockey.index == 520130914].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
520130914,2019-04-16 18:21:14,2019-04-16 19:19:46,00:58:32,2019-04-16 18:22:04,Tex Steak Sand,2.0,0,0,0.0,1.0,15.99,18.069318,5
520130914,2019-04-16 18:21:14,2019-04-16 19:19:46,00:58:32,2019-04-16 18:22:04,K- Chic Fingers,2.0,0,0,0.0,1.0,8.99,10.159047,5
520130914,2019-04-16 18:21:14,2019-04-16 19:19:46,00:58:32,2019-04-16 18:22:04,1/2 Chk Fajita,2.0,0,0,0.0,1.0,19.99,22.589473,5
520130914,2019-04-16 18:21:14,2019-04-16 19:19:46,00:58:32,2019-04-16 18:22:04,Greek LG,2.0,0,0,0.0,1.0,17.48,19.753076,5
520130914,2019-04-16 18:21:14,2019-04-16 19:19:46,00:58:32,2019-04-16 18:22:04,K- Spag&Meatball,2.0,0,0,0.0,1.0,9.99,11.289086,5
520130914,2019-04-16 18:21:14,2019-04-16 19:19:46,00:58:32,2019-04-16 18:22:04,*No Bev,4.0,0,0,0.0,1.0,0.00,0.000000,5


`Family Event`

#### 11. Table 523514188

In [80]:
hockey[hockey.index == 523514188].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 16:54:38,BUTCHER BLOCK,2.0,0,0,0.0,1.0,18.00,20.3400,3
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 16:54:38,OVEN- BRAISED BRISKET SANDWICH,2.0,0,0,0.0,1.0,16.00,18.0800,3
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 16:54:38,PRIMAVERA,2.0,0,0,0.0,1.0,14.27,16.1251,3
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 17:49:02,GLS MALBEC ANDELUA ARGN,6.0,0,0,0.0,1.0,16.00,18.0800,3
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 17:49:02,GLS CHARDONNY LOST ANGEL CAL,6.0,0,0,0.0,1.0,16.00,18.0800,3
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 17:49:02,GLS Pinot Grigio IGT,6.0,0,0,0.0,1.0,11.41,12.8933,3
523514188,2019-04-20 16:54:00,2019-04-20 17:52:20,00:58:20,2019-04-20 17:49:50,BEEFEATER,3.0,0,0,0.0,2.0,12.32,13.9216,3


`Dinner`

#### 12. Table 523328731

In [81]:
hockey[hockey.index == 523328731].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
523328731,2019-04-20 13:34:27,2019-04-20 14:44:32,01:10:05,2019-04-20 13:34:34,Sweet Fries APP,2.0,0,0,0.0,1.0,7.99,9.028256,4
523328731,2019-04-20 13:34:27,2019-04-20 14:44:32,01:10:05,2019-04-20 13:35:15,Bacon & Eggs,2.0,0,0,0.0,1.0,10.99,12.418089,4
523328731,2019-04-20 13:34:27,2019-04-20 14:44:32,01:10:05,2019-04-20 13:35:15,PB House Burger,2.0,0,0,0.0,1.0,14.99,16.937867,4
523328731,2019-04-20 13:34:27,2019-04-20 14:44:32,01:10:05,2019-04-20 13:35:15,Grill Cheeseburg,2.0,0,0,0.0,1.0,15.99,18.067811,4
523328731,2019-04-20 13:34:27,2019-04-20 14:44:32,01:10:05,2019-04-20 13:35:15,Classic Burger,2.0,0,0,0.0,1.0,12.99,14.677978,4


`Lunch`

#### 13. Table 521804862

In [82]:
hockey[hockey.index == 521804862].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:48:54,Crispy Calamari,2.0,0,0,0.0,1.0,13.99,15.808909,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:48:54,Coconut Shrimp,3.0,0,0,0.0,1.0,11.99,13.548879,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:48:54,Fount Pop,4.0,0,0,0.0,2.0,5.98,6.757489,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:55:06,Kung Pao Chicken,2.0,0,0,0.0,1.0,16.99,19.198954,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:55:06,Lobster Jumbo,2.0,0,0,0.0,1.0,21.99,24.849028,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:55:06,Spanish Paella,2.0,0,0,0.0,1.0,17.99,20.328968,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 19:55:06,Maple Nut Salmon,2.0,0,0,0.0,1.0,19.99,22.588998,4
521804862,2019-04-18 19:48:42,2019-04-18 21:34:24,01:45:42,2019-04-18 20:54:05,Killer Shake,4.0,0,0,0.0,1.0,4.99,5.638774,4


`Dinner`

#### 14. Table 524442000

In [83]:
hockey[hockey.index == 524442000].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:11:43,14oz Organic,1.0,350,2,0.414,1.0,6.75,7.627282,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:11:43,Guinness,1.0,26,3,0.440,1.0,7.50,8.474758,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:11:43,Perrier LG,4.0,0,0,0.000,1.0,4.49,5.073555,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:25:47,SUN- 14oz Stella,1.0,44,2,0.414,1.0,6.50,7.344790,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:26:38,Pesto Chicken,2.0,0,0,0.000,1.0,18.99,21.458086,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:26:38,10oz Steak Frite,2.0,0,0,0.000,1.0,25.99,29.367860,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:26:38,PB Jumbo Club,2.0,0,0,0.000,1.0,17.97,20.305519,4
524442000,2019-04-21 18:11:19,2019-04-21 20:07:07,01:55:48,2019-04-21 18:26:38,Fettuccine Pomo,2.0,0,0,0.000,1.0,16.99,19.198151,4


`Dinner`

#### 15. Table 514472097

In [84]:
hockey[hockey.index == 514472097].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
514472097,2019-04-13 15:04:00,2019-04-13 16:16:45,01:12:45,2019-04-13 15:08:00,CHEESE BREAD,2.0,0,0,0.0,1.0,5.88,6.644369,3
514472097,2019-04-13 15:04:00,2019-04-13 16:16:45,01:12:45,2019-04-13 15:08:00,KIDS-TENDERS,2.0,0,0,0.0,1.0,9.89,11.175649,3
514472097,2019-04-13 15:04:00,2019-04-13 16:16:45,01:12:45,2019-04-13 15:08:00,CHICKEN CAESAR,2.0,0,0,0.0,1.0,12.88,14.554333,3
514472097,2019-04-13 15:04:00,2019-04-13 16:16:45,01:12:45,2019-04-13 15:08:00,KIDS-TENDERS,2.0,0,0,0.0,1.0,9.89,11.175649,3


`Family Event`

#### 16. Table 521770033

In [85]:
hockey[hockey.index == 521770033].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 17:45:03,BELGIAN.MOON..XL,1.0,84,2,0.591,1.0,10.01,11.311343,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 17:45:03,BELGIAN.MOON..Pt,1.0,84,2,0.473,1.0,8.43,9.525936,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 17:56:55,Add.Caesar--,5.0,0,0,0.000,1.0,0.00,0.000000,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 17:56:55,CHK.GOATCHS.WRAP,2.0,0,0,0.000,1.0,15.99,18.068768,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 17:56:55,CHK.SAND.SAUCD,2.0,0,0,0.000,1.0,17.59,19.876775,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 18:36:26,BELGIAN.MOON..Pt,1.0,84,2,0.473,1.0,8.43,9.525936,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 18:36:26,WATER.,4.0,0,0,0.000,1.0,0.00,0.000000,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 19:18:25,n/c.sauce,2.0,0,0,0.000,1.0,0.00,0.000000,3
521770033,2019-04-18 17:45:00,2019-04-18 20:50:22,03:05:22,2019-04-18 19:18:25,w.hot sauce,2.0,0,0,0.000,1.0,0.00,0.000000,3


`Dinner`

#### 17. Table 520119333

In [86]:
hockey[hockey.index == 520119333].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,ED Starter,2.0,0,0,0.0,1.0,0.0,0.00,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,ED Salmon,2.0,0,0,0.0,1.0,18.0,20.34,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,ED Chicken Curry,2.0,0,0,0.0,1.0,18.0,20.34,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,ED Sirloin,2.0,0,0,0.0,1.0,18.0,20.34,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,ED Starter,2.0,0,0,0.0,1.0,0.0,0.00,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,ED Starter,2.0,0,0,0.0,1.0,0.0,0.00,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:10:12,****COURSE 2****,2.0,0,0,0.0,1.0,0.0,0.00,3
520119333,2019-04-16 17:07:02,2019-04-16 18:12:51,01:05:49,2019-04-16 17:25:51,PICK UP,5.0,0,0,0.0,1.0,0.0,0.00,3


`Dinner`

#### 18. Table 525397783

In [87]:
hockey[hockey.index == 525397783].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
525397783,2019-04-23 12:08:00,2019-04-23 13:00:22,00:52:22,2019-04-23 12:08:30,DAILY.CURRY,2.0,0,0,0.0,1.0,0.00,0.000000,3
525397783,2019-04-23 12:08:00,2019-04-23 13:00:22,00:52:22,2019-04-23 12:08:30,CHICKEN.TIKKA,2.0,0,0,0.0,1.0,21.95,24.805766,3
525397783,2019-04-23 12:08:00,2019-04-23 13:00:22,00:52:22,2019-04-23 12:08:30,LOBSTR.GRL.CHS,2.0,0,0,0.0,1.0,21.75,24.579746,3
525397783,2019-04-23 12:08:00,2019-04-23 13:00:22,00:52:22,2019-04-23 12:08:30,STEAK.PIE,2.0,0,0,0.0,1.0,19.25,21.754488,3


`Lunch`

#### 19. Table 511353667

In [88]:
hockey[hockey.index == 511353667].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
511353667,2019-04-10 16:44:20,2019-04-10 18:03:39,01:19:19,2019-04-10 16:44:33,French 75,3.0,0,0,0.0,1.0,16.0,18.08,3
511353667,2019-04-10 16:44:20,2019-04-10 18:03:39,01:19:19,2019-04-10 16:44:33,Cassidy,3.0,0,0,0.0,1.0,18.0,20.34,3
511353667,2019-04-10 16:44:20,2019-04-10 18:03:39,01:19:19,2019-04-10 16:44:33,French 75,3.0,0,0,0.0,1.0,16.0,18.08,3
511353667,2019-04-10 16:44:20,2019-04-10 18:03:39,01:19:19,2019-04-10 17:03:20,Charc & Cheese,2.0,0,0,0.0,1.0,21.0,23.73,3
511353667,2019-04-10 16:44:20,2019-04-10 18:03:39,01:19:19,2019-04-10 18:03:39,Member Account,5.0,0,0,0.0,1.0,0.0,0.00,3


`After Work`

#### 20. Table 522420188

In [89]:
hockey[hockey.index == 522420188].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
522420188,2019-04-19 15:23:00,2019-04-19 15:59:08,00:36:08,2019-04-19 15:23:10,BREWSKY.RT.BEER,4.0,0,0,0.000,1.0,4.45,5.029045,3
522420188,2019-04-19 15:23:00,2019-04-19 15:59:08,00:36:08,2019-04-19 15:23:10,STELLA.Pt,1.0,44,2,0.591,1.0,9.33,10.544042,3
522420188,2019-04-19 15:23:00,2019-04-19 15:59:08,00:36:08,2019-04-19 15:26:09,CHEESE.CHIKN.MAC,2.0,0,0,0.000,1.0,19.97,22.568544,3
522420188,2019-04-19 15:23:00,2019-04-19 15:59:08,00:36:08,2019-04-19 15:26:09,SHRIMP.PASTA,2.0,0,0,0.000,1.0,20.47,23.133606,3
522420188,2019-04-19 15:23:00,2019-04-19 15:59:08,00:36:08,2019-04-19 15:26:09,HOLY.SMOKE.BURGR,2.0,0,0,0.000,1.0,17.97,20.308300,3
522420188,2019-04-19 15:23:00,2019-04-19 15:59:08,00:36:08,2019-04-19 15:49:13,S/O.Fries,2.0,0,0,0.000,1.0,2.97,3.356464,3


`Lunch`

### Valentine

#### 1. Table 446798723

In [90]:
valentine[valentine.index == 446798723].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
446798723,2019-02-12 18:23:00,2019-02-12 19:17:09,00:54:09,2019-02-12 18:23:00,ORGANIC TEA,4.0,0,0,0.0,1.0,3.00,3.390041,3
446798723,2019-02-12 18:23:00,2019-02-12 19:17:09,00:54:09,2019-02-12 18:23:00,COKE,4.0,0,0,0.0,1.0,3.00,3.390041,3
446798723,2019-02-12 18:23:00,2019-02-12 19:17:09,00:54:09,2019-02-12 18:29:00,W.COAST BURGER,2.0,0,0,0.0,1.0,17.43,19.696137,3
446798723,2019-02-12 18:23:00,2019-02-12 19:17:09,00:54:09,2019-02-12 18:29:00,BBQ CHK SALAD,2.0,0,0,0.0,1.0,19.86,22.442071,3
446798723,2019-02-12 18:23:00,2019-02-12 19:17:09,00:54:09,2019-02-12 18:29:00,SIERRA SALMON,2.0,0,0,0.0,1.0,22.78,25.741710,3


`Dinner`

#### 2. Table 448169651

In [91]:
valentine[valentine.index == 448169651].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:10:00,LG STRONGBOW,1.0,111,2,0.591,1.0,7.75,8.757426,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:10:00,LG CANADIAN,1.0,183,2,0.591,1.0,7.25,8.192431,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:10:00,LG CANADIAN,1.0,183,2,0.591,1.0,7.25,8.192431,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:17:00,LRG RICKARDS,1.0,191,2,0.591,1.0,7.75,8.757426,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:17:00,LG STELLA,1.0,44,2,0.591,1.0,8.75,9.887417,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:39:00,JUG BANQUET,1.0,1120,2,1.774,1.0,22.50,25.424786,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 15:39:00,JUG BANQUET,1.0,1120,2,1.774,1.0,22.50,25.424786,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 16:00:00,CHICKEN FINGERS,2.0,0,0,0.000,1.0,6.00,6.779943,4
448169651,2019-02-15 15:10:00,2019-02-15 19:47:31,04:37:31,2019-02-15 16:00:00,GARLICBR/CHEESE,2.0,0,0,0.000,1.0,4.00,4.519962,4


`Social Gathering`

#### 3. Table 434708231

In [92]:
valentine[valentine.index == 434708231].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:11:00,LG BUDWEISER,1.0,71,2,0.591,1.0,7.25,8.192934,3
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:11:00,LRG RICKARDS,1.0,191,2,0.591,1.0,7.75,8.757964,3
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:25:00,TWELVE WINGS,2.0,0,0,0.000,1.0,20.50,23.166228,3
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:25:00,JERK CHICKEN,2.0,0,0,0.000,1.0,18.00,20.341078,3
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:25:00,FISH & CHIPS,2.0,0,0,0.000,1.0,15.00,16.950898,3
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:40:00,LG BUDWEISER,1.0,71,2,0.591,1.0,7.25,8.192934,3
434708231,2019-02-10 19:11:00,2019-02-10 20:18:07,01:07:07,2019-02-10 19:53:00,LRG RICKARDS,1.0,191,2,0.591,1.0,7.75,8.757964,3


`Dinner` (/ `Social Gathering`)

#### 4. Table 447647574

In [93]:
valentine[valentine.index == 447647574].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-14 23:54:00,WELL GIN,3.0,0,0,0.0,1.0,5.95,6.724253,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-14 23:54:00,WELL VODKA,3.0,0,0,0.0,1.0,5.95,6.724253,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-14 23:54:00,WELL VODKA DBL,3.0,0,0,0.0,1.0,10.25,11.583797,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-14 23:54:00,PB&J,3.0,0,0,0.0,1.0,11.75,13.278987,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-14 23:54:00,FIREBALL SHOT,3.0,0,0,0.0,1.0,5.95,6.724253,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-14 23:58:00,WELL GIN,3.0,0,0,0.0,1.0,5.95,6.724253,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-15 00:16:00,FIREBALL SHOT,3.0,0,0,0.0,1.0,5.95,6.724253,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-15 00:29:00,JAGER SHOT,3.0,0,0,0.0,1.0,6.25,7.063291,4
447647574,2019-02-14 23:54:00,2019-02-15 01:18:20,01:24:20,2019-02-15 00:29:00,JAGER SHOT,3.0,0,0,0.0,1.0,6.25,7.063291,4


`Drinking`

#### 5. Table 448003813

In [94]:
valentine[valentine.index == 448003813].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 17:56:00,KIDBEV,4.0,0,0,0.000,1.0,0.00,0.000000,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 17:56:00,CROWN.ROYAL*,3.0,0,0,0.000,1.0,7.25,8.192595,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 17:56:00,BREWSKY.RT.BEER,4.0,0,0,0.000,1.0,4.45,5.028558,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 17:56:00,BELGIAN.MOON..Pt,1.0,84,2,0.473,1.0,8.43,9.526011,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 18:05:00,SESAME CHICKEN,2.0,0,0,0.000,1.0,20.25,22.882766,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 18:05:00,FAJ.CHICKEN,2.0,0,0,0.000,1.0,21.81,24.645586,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 18:05:00,CHK.PARMIGIANA,2.0,0,0,0.000,1.0,20.87,23.583374,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 18:05:00,FAJ.FOR.1,2.0,0,0,0.000,1.0,0.00,0.000000,4
448003813,2019-02-15 17:56:00,2019-02-15 19:18:01,01:22:01,2019-02-15 18:05:00,HOLY.SMOKE.BURGR,2.0,0,0,0.000,1.0,17.97,20.306336,4


`Family Event`

#### 6. Table 447986271

In [95]:
valentine[valentine.index == 447986271].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 16:52:00,RICKARD.RED.XL,1.0,191,2,0.591,1.0,10.01,11.311356,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 16:52:00,KIDBEV,4.0,0,0,0.000,1.0,0.00,0.000000,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:01:00,MEXI.SALAD,2.0,0,0,0.000,1.0,18.99,21.458807,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:01:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:01:00,FINGERS.SAUCED,2.0,0,0,0.000,1.0,16.87,19.063195,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:01:00,POUTINE,2.0,0,0,0.000,1.0,0.00,0.000000,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:01:00,POUT.CLASSIC,2.0,0,0,0.000,1.0,9.53,10.768954,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:01:00,K.FISH.STIX,2.0,0,0,0.000,1.0,8.98,10.147451,3
447986271,2019-02-15 16:52:00,2019-02-15 18:12:04,01:20:04,2019-02-15 17:19:00,R.JT.RS.CS/CF.Gl,6.0,0,0,0.000,1.0,6.54,7.390237,3


`Family Event`

#### 7. Table 447020385

In [96]:
valentine[valentine.index == 447020385].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 12:23:00,DAILY.FISH,2.0,0,0,0.0,1.0,0.00,0.000000,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 12:23:00,DC.LINGCOD,2.0,0,0,0.0,1.0,29.95,33.843644,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 12:23:00,DC.LINGCOD,2.0,0,0,0.0,1.0,29.95,33.843644,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 12:23:00,DAILY.FISH,2.0,0,0,0.0,1.0,0.00,0.000000,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 12:23:00,DAILY.FISH,2.0,0,0,0.0,1.0,0.00,0.000000,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 12:23:00,DC.LINGCOD,2.0,0,0,0.0,1.0,29.95,33.843644,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 13:14:00,CAFE.LATTE,4.0,0,0,0.0,1.0,4.95,5.593524,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 13:14:00,CAFE.LATTE,4.0,0,0,0.0,1.0,4.95,5.593524,3
447020385,2019-02-13 12:23:00,2019-02-13 13:29:39,01:06:39,2019-02-13 13:14:00,AMERICANO,4.0,0,0,0.0,1.0,4.40,4.972021,3


`Lunch`

#### 8. Table 447091085

In [97]:
valentine[valentine.index == 447091085].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:17:00,SAPPORO.DRF.Mg,1.0,270,2,0.591,1.0,8.00,9.039921,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:17:00,SAPPORO.DRF.Mg,1.0,270,2,0.591,1.0,8.00,9.039921,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:17:00,TANQUERAY*,3.0,0,0,0.000,1.0,7.25,8.192428,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:36:00,SAPPORO.DRF.Mg,1.0,270,2,0.591,1.0,8.00,9.039921,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:36:00,SLIDERS+,2.0,0,0,0.000,1.0,15.50,17.514846,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:36:00,BANGKOK.NACHOS,2.0,0,0,0.000,1.0,15.50,17.514846,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:54:00,TANQUERAY*,3.0,0,0,0.000,1.0,7.25,8.192428,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 18:54:00,SAPPORO.DRF.Mg,1.0,270,2,0.591,1.0,8.00,9.039921,4
447091085,2019-02-13 18:17:00,2019-02-13 20:23:58,02:06:58,2019-02-13 19:23:00,STELLA.Mg,3.0,0,0,0.000,1.0,9.50,10.734906,4


`Dinner`

#### 9. Table 435006071

In [98]:
valentine[valentine.index == 435006071].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:24:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:24:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:24:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:24:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:35:00,Add.Caesar--,5.0,0,0,0.0,1.0,0.00,0.000000,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:35:00,VEGGIE.BURGER,2.0,0,0,0.0,1.0,17.62,19.911410,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:35:00,DBL.BAC.CHSBRG,2.0,0,0,0.0,1.0,17.73,20.035715,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:35:00,ADD.GUAC*,2.0,0,0,0.0,1.0,2.99,3.378837,3
435006071,2019-02-11 12:24:00,2019-02-11 13:18:52,00:54:52,2019-02-11 12:35:00,S/O.Bf.Gravy,2.0,0,0,0.0,1.0,1.49,1.683768,3


`Lunch`

#### 10. Table 434752383

In [99]:
valentine[valentine.index == 434752383].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434752383,2019-02-10 22:12:00,2019-02-10 22:59:24,00:47:24,2019-02-10 22:12:00,LG COORS LIGHT,1.0,280,2,0.591,1.0,7.25,8.191929,4
434752383,2019-02-10 22:12:00,2019-02-10 22:59:24,00:47:24,2019-02-10 22:12:00,LG GUINNESS,1.0,26,2,0.591,1.0,8.75,9.886811,4
434752383,2019-02-10 22:12:00,2019-02-10 22:59:24,00:47:24,2019-02-10 22:12:00,CORONA,1.0,128,3,0.473,1.0,7.35,8.304921,4
434752383,2019-02-10 22:12:00,2019-02-10 22:59:24,00:47:24,2019-02-10 22:12:00,LG STELLA,1.0,44,2,0.591,1.0,8.75,9.886811,4
434752383,2019-02-10 22:12:00,2019-02-10 22:59:24,00:47:24,2019-02-10 22:12:00,MILLER GEN,1.0,930,1,0.355,1.0,6.00,6.779528,4


`Social Gathering`

#### 11. Table 447523773

In [100]:
valentine[valentine.index == 447523773].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:35:00,PTCH-BLUSHING SANGRIA,6.0,0,0,0.00,1.0,30.00,33.899936,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:35:00,VODKA-FINLDIA,3.0,0,0,0.00,1.0,6.50,7.344986,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:36:00,PLAIN GARLIC BRD,2.0,0,0,0.00,1.0,3.00,3.389994,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:46:00,CUP SOUP,2.0,0,0,0.00,1.0,0.00,0.000000,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:46:00,CALAMARI,2.0,0,0,0.00,1.0,13.67,15.447071,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:46:00,LOBSTER MAC&CH,2.0,0,0,0.00,1.0,33.00,37.289930,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:46:00,KIDS-PEN.TOM,2.0,0,0,0.00,1.0,9.89,11.175679,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:46:00,FRENCH FRIES,2.0,0,0,0.00,1.0,6.97,7.876085,5
447523773,2019-02-14 20:35:00,2019-02-14 21:42:59,01:07:59,2019-02-14 20:46:00,NY STRIP-VDAY,2.0,0,0,0.00,1.0,33.00,37.289930,5


`Family`

#### 12. Table 435067253

In [101]:
valentine[valentine.index == 435067253].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:43:00,R.CLIF.CB.SHZ.1z,6.0,0,0,0.000,1.0,14.00,15.819766,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:43:00,TOP.SHLF.MARG,3.0,0,0,0.000,1.0,11.00,12.429816,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:43:00,BUD LIGHT BTL,1.0,15,1,0.341,1.0,6.75,7.627387,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:43:00,POP.,4.0,0,0,0.000,1.0,3.99,4.508633,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:49:00,b.potato,2.0,0,0,0.000,1.0,0.00,0.000000,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:49:00,b.potato,2.0,0,0,0.000,1.0,0.00,0.000000,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:49:00,MOROCCAN.CHICKEN*,2.0,0,0,0.000,1.0,24.50,27.684591,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:49:00,$Truffle.Fries$$,2.0,0,0,0.000,1.0,3.95,4.463434,4
435067253,2019-02-11 18:43:00,2019-02-11 20:02:47,01:19:47,2019-02-11 18:49:00,CHEESE.BURGER,2.0,0,0,0.000,1.0,17.95,20.283200,4


`Dinner`

#### 13. Table 434617535

In [102]:
valentine[valentine.index == 434617535].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
434617535,2019-02-10 15:20:00,2019-02-10 15:48:02,00:28:02,2019-02-10 15:20:04,CALL ME DARK,3.0,0,0,0.0,1.0,8.25,9.320353,4
434617535,2019-02-10 15:20:00,2019-02-10 15:48:02,00:28:02,2019-02-10 15:20:04,BLUE HAWAII,3.0,0,0,0.0,1.0,6.75,7.625743,4
434617535,2019-02-10 15:20:00,2019-02-10 15:48:02,00:28:02,2019-02-10 15:20:04,FIREBALL,3.0,0,0,0.0,1.0,5.95,6.721952,4
434617535,2019-02-10 15:20:00,2019-02-10 15:48:02,00:28:02,2019-02-10 15:20:04,FIREBALL,3.0,0,0,0.0,1.0,5.95,6.721952,4


`Drinking`

#### 14. Table 447986377

In [103]:
valentine[valentine.index == 447986377].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,Crispy Calamari,2.0,0,0,0.0,0.33,4.266887,4.821561,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,Fount Pop,4.0,0,0,0.0,1.00,2.737766,3.093662,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,Crispy Calamari,2.0,0,0,0.0,0.33,4.266887,4.821561,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,Fount Pop,4.0,0,0,0.0,1.00,2.737766,3.093662,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,Fount Pop,4.0,0,0,0.0,1.00,2.737766,3.093662,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,PB House Burger,2.0,0,0,0.0,1.00,21.929599,24.780339,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,PB House Burger,2.0,0,0,0.0,1.00,16.463223,18.603361,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,PB House Burger,2.0,0,0,0.0,1.00,19.200989,21.697023,3
447986377,2019-02-15 18:00:50,2019-02-15 19:03:46,01:02:56,2019-02-15 18:01:34,Crispy Calamari,2.0,0,0,0.0,0.33,4.276043,4.831907,3


`Dinner`

#### 15. Table 447877104

In [104]:
valentine[valentine.index == 447877104].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:06:23,Main Beet Salad,2.0,0,0,0.0,1.0,23.0,25.991243,3
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:06:23,Butter Chicken,2.0,0,0,0.0,1.0,27.0,30.511459,3
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:06:23,Butter Chicken,2.0,0,0,0.0,1.0,27.0,30.511459,3
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:06:23,Potato Chips,2.0,0,0,0.0,1.0,8.0,9.040432,3
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:06:35,Pop,4.0,0,0,0.0,1.0,2.5,2.825135,3
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:16:56,Pop,4.0,0,0,0.0,1.0,2.5,2.825135,3
447877104,2019-02-15 12:05:41,2019-02-15 12:49:28,00:43:47,2019-02-15 12:16:56,Pop,4.0,0,0,0.0,1.0,2.5,2.825135,3


`Lunch`

#### 16. Table 447558662

In [105]:
valentine[valentine.index == 447558662].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:36:00,SAMBUCA*,3.0,0,0,0.000,2.0,12.69,14.340203,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:36:00,POLAR.ICE*,3.0,0,0,0.000,2.0,12.25,13.842985,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:36:00,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:38:00,COORS.LIGHT BTL,1.0,280,1,0.340,1.0,5.96,6.735036,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:43:00,DBL.BAC.CHSBRG,2.0,0,0,0.000,1.0,17.73,20.035602,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:43:00,+ADD.TUNA,2.0,0,0,0.000,1.0,4.99,5.638898,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:43:00,TUNA.SALAD+,2.0,0,0,0.000,1.0,19.87,22.453887,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:43:00,TUNA.SALAD+,2.0,0,0,0.000,1.0,19.87,22.453887,3
447558662,2019-02-14 20:36:00,2019-02-14 22:48:46,02:12:46,2019-02-14 20:43:00,+ADD.TUNA,2.0,0,0,0.000,1.0,4.99,5.638898,3


`Family Event`

#### 17. Table 448008007

In [106]:
valentine[valentine.index == 448008007].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 17:56:00,ORGANIC TEA,4.0,0,0,0.0,1.0,3.00,3.390036,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 18:09:00,BBQ RIBLETS,2.0,0,0,0.0,1.0,18.48,20.882619,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 18:09:00,MARIA s PASTA,2.0,0,0,0.0,1.0,16.28,18.396593,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 18:09:00,CLASSIC BURGER,2.0,0,0,0.0,1.0,13.27,14.995257,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 18:09:00,CLASSIC BURGER,2.0,0,0,0.0,1.0,13.27,14.995257,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 19:06:00,FUDGE SUNDAE,2.0,0,0,0.0,1.0,8.00,9.040095,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 19:06:00,KIDS ICE CREAM $2,2.0,0,0,0.0,1.0,2.00,2.260024,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 19:06:00,STRAW CH CAKE,2.0,0,0,0.0,1.0,7.00,7.910083,4
448008007,2019-02-15 17:56:00,2019-02-15 19:35:03,01:39:03,2019-02-15 19:15:00,ORGANIC TEA,4.0,0,0,0.0,1.0,3.00,3.390036,4


`Family Event`

#### 18. Table 447497220

In [107]:
valentine[valentine.index == 447497220].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447497220,2019-02-14 20:26:00,2019-02-14 20:59:33,00:33:33,2019-02-14 20:26:00,GREY GOOSE,3.0,0,0,0.0,1.0,8.5,9.606809,3
447497220,2019-02-14 20:26:00,2019-02-14 20:59:33,00:33:33,2019-02-14 20:48:00,GREY GOOSE,3.0,0,0,0.0,1.0,8.5,9.606809,3
447497220,2019-02-14 20:26:00,2019-02-14 20:59:33,00:33:33,2019-02-14 20:48:00,RED BULL,4.0,0,0,0.0,1.0,6.5,7.346383,3


`Drinking`

#### 19. Table 435171863

In [108]:
valentine[valentine.index == 435171863].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-11 23:59:00,Jacks.Veg+,2.0,0,0,0.0,1.0,0.00,0.000000,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-11 23:59:00,NY.MEMPHIS+,2.0,0,0,0.0,1.0,33.09,37.391526,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-11 23:59:00,Mashed,2.0,0,0,0.0,1.0,0.00,0.000000,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-11 23:59:00,Add.Soup^,2.0,0,0,0.0,1.0,5.94,6.712169,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-11 23:59:00,SHRIMP.PASTA,2.0,0,0,0.0,1.0,20.47,23.130992,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-12 00:01:00,BLOODY.CAESAR,3.0,0,0,0.0,1.0,8.97,10.136053,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-12 00:01:00,POLAR.ICE..,3.0,0,0,0.0,1.0,0.00,0.000000,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-12 00:05:00,ADD PAN W/CHS,2.0,0,0,0.0,1.0,6.39,7.220666,3
435171863,2019-02-11 23:59:00,2019-02-12 01:06:29,01:07:29,2019-02-12 00:15:00,ASIAGO.BOWTIE,2.0,0,0,0.0,1.0,19.99,22.588595,3


`Dinner`

#### 20. Table 447019409

In [109]:
valentine[valentine.index == 447019409].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 12:23:17,Fish & Chips,2.0,0,0,0.0,1.0,13.99,15.808520,4
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 12:23:17,Classic Burger,2.0,0,0,0.0,1.0,12.99,14.678533,4
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 12:23:17,Mexican Chicken,2.0,0,0,0.0,1.0,16.99,19.198481,4
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 12:23:17,Fettuccine Pomo,2.0,0,0,0.0,1.0,16.99,19.198481,4
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 13:09:02,Juice,4.0,0,0,0.0,1.0,2.99,3.378662,4
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 13:09:02,Juice,4.0,0,0,0.0,1.0,2.99,3.378662,4
447019409,2019-02-13 12:22:40,2019-02-13 13:13:19,00:50:39,2019-02-13 13:09:02,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378662,4


`Lunch`

### Silvester

#### 1. Table 363177655

In [110]:
silvester[silvester.index == 363177655].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
363177655,2019-01-03 09:31:29,2019-01-03 10:04:37,00:33:08,2019-01-03 09:31:56,Early Bird,2.0,0,0,0.0,1.0,7.49,8.463333,3
363177655,2019-01-03 09:31:29,2019-01-03 10:04:37,00:33:08,2019-01-03 09:31:56,Early Bird,2.0,0,0,0.0,1.0,7.49,8.463333,3
363177655,2019-01-03 09:31:29,2019-01-03 10:04:37,00:33:08,2019-01-03 09:31:56,Early Bird,2.0,0,0,0.0,1.0,7.49,8.463333,3


`Breakfast`

#### 2. Table 359978168

In [111]:
silvester[silvester.index == 359978168].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,Great Reuben,2.0,0,0,0.0,1.0,14.99,16.939015,5
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,P.B Deli Sand,2.0,0,0,0.0,1.0,15.98,18.057736,5
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,Pesto Chicken,2.0,0,0,0.0,1.0,18.99,21.459099,5
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,K- Choc Pancakes,2.0,0,0,0.0,1.0,8.99,10.158889,5
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378763,5
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378763,5
359978168,2018-12-26 14:04:09,2018-12-26 15:18:16,01:14:07,2018-12-26 14:05:24,P.B Deli Sand,2.0,0,0,0.0,1.0,15.98,18.057736,5


`Family Event`

#### 3. Table 359644089

In [112]:
silvester[silvester.index == 359644089].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 20:58:00,THE MAILMAN,3.0,0,0,0.0,1.0,7.25,8.137830,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 20:58:00,MOSCOW MULE,3.0,0,0,0.0,1.0,9.75,10.943979,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 20:58:00,TACO BAJA SHRIMP,2.0,0,0,0.0,1.0,5.95,6.678633,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 20:58:00,THAI CHK WRAP,2.0,0,0,0.0,1.0,8.20,9.204167,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 20:58:00,NACHOS,2.0,0,0,0.0,1.0,7.70,8.642937,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 20:58:00,SOUTHWEST SALAD,2.0,0,0,0.0,1.0,5.95,6.678633,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 21:22:00,THE MAILMAN,3.0,0,0,0.0,1.0,7.25,8.137830,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 21:22:00,MOSCOW MULE,3.0,0,0,0.0,1.0,9.75,10.943979,3
359644089,2018-12-25 20:58:00,2018-12-25 21:49:05,00:51:05,2018-12-25 21:22:00,TACO BAJA SHRIMP,2.0,0,0,0.0,1.0,5.95,6.678633,3


`Social Gathering`

#### 4. Table 362094000

In [113]:
silvester[silvester.index == 362094000].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
362094000,2018-12-31 10:18:39,2018-12-31 11:21:10,01:02:31,2018-12-31 10:19:24,Create Omelette,2.0,0,0,0.0,1.0,12.99,14.677523,4
362094000,2018-12-31 10:18:39,2018-12-31 11:21:10,01:02:31,2018-12-31 10:19:24,Early Bird,2.0,0,0,0.0,1.0,7.49,8.463021,4
362094000,2018-12-31 10:18:39,2018-12-31 11:21:10,01:02:31,2018-12-31 10:19:24,Create Omelette,2.0,0,0,0.0,1.0,12.99,14.677523,4
362094000,2018-12-31 10:18:39,2018-12-31 11:21:10,01:02:31,2018-12-31 10:19:24,Early Bird,2.0,0,0,0.0,1.0,7.49,8.463021,4
362094000,2018-12-31 10:18:39,2018-12-31 11:21:10,01:02:31,2018-12-31 10:19:24,Coffee,4.0,0,0,0.0,1.0,2.69,3.039456,4
362094000,2018-12-31 10:18:39,2018-12-31 11:21:10,01:02:31,2018-12-31 10:19:24,Coffee,4.0,0,0,0.0,1.0,2.69,3.039456,4


`Breakfast`

#### 5. Table 361772528

In [114]:
silvester[silvester.index == 361772528].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:37:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508688,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:37:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508688,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:48:00,b.potato,2.0,0,0,0.0,1.0,0.00,0.000000,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:48:00,CAESAR.APP,2.0,0,0,0.0,1.0,8.95,10.113473,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:48:00,10OZ.PRIME.RIB,2.0,0,0,0.0,1.0,32.45,36.668402,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:48:00,CHK.PENNE*,2.0,0,0,0.0,1.0,20.95,23.673437,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:48:00,GARDEN.APP,2.0,0,0,0.0,1.0,8.95,10.113473,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 18:48:00,PAN.RST.SALMON*,2.0,0,0,0.0,1.0,29.95,33.843409,3
361772528,2018-12-30 18:37:00,2018-12-30 20:18:08,01:41:08,2018-12-30 19:52:00,MANGO.SORBET,2.0,0,0,0.0,1.0,5.95,6.723482,3


`Dinner`

#### 6. Table 359978861

In [115]:
silvester[silvester.index == 359978861].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,Mashed,2.0,0,0,0.0,1.0,0.00,0.000000,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,Jacks.Veg+,2.0,0,0,0.0,1.0,0.00,0.000000,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,SIRL.MEMPHIS+,2.0,0,0,0.0,1.0,27.50,31.075037,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,STEAK,2.0,0,0,0.0,1.0,0.00,0.000000,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,cajun.shrimp,2.0,0,0,0.0,1.0,0.00,0.000000,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,QUESADILLAS,2.0,0,0,0.0,1.0,14.97,16.916120,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,SIRL.BOURBON+,2.0,0,0,0.0,1.0,27.77,31.380137,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,Jacks.Veg+,2.0,0,0,0.0,1.0,0.00,0.000000,3
359978861,2018-12-26 15:17:00,2018-12-26 15:50:02,00:33:02,2018-12-26 15:17:00,STEAK,2.0,0,0,0.0,1.0,0.00,0.000000,3


`Lunch`

#### 7. Table 360862630

In [116]:
silvester[silvester.index == 360862630].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 17:48:28,BUDWEISER TAP,1.0,71,2,0.414,1.0,6.00,6.731803,3
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 18:21:41,WORKS BURGER,2.0,0,0,0.000,1.0,7.90,8.863541,3
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 18:21:41,POTATO SKINS,2.0,0,0,0.000,1.0,5.95,6.675705,3
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 18:21:41,WINGS - HOT,2.0,0,0,0.000,1.0,6.90,7.741574,3
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 18:22:08,DAM AMBER TAP,1.0,6186,2,0.414,1.0,6.50,7.292787,3
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 18:23:28,DAM AMBER TAP,1.0,6186,2,0.414,1.0,6.50,7.292787,3
360862630,2018-12-28 18:21:00,2018-12-28 19:09:36,00:48:36,2018-12-28 18:23:28,BUDWEISER TAP,1.0,71,2,0.414,1.0,6.00,6.731803,3


`Social Gathering`

#### 8. Table 362626040

In [117]:
silvester[silvester.index == 362626040].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 15:44:50,GENUIN PLUM,3.0,0,0,0.0,1.0,12.75,14.359151,4
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 15:44:50,GENUIN PLUM,3.0,0,0,0.0,1.0,12.75,14.359151,4
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 16:05:39,BAJA FISH TACOS,2.0,0,0,0.0,1.0,5.95,6.700937,4
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 16:05:39,PENNE SUNDRIED,2.0,0,0,0.0,1.0,5.95,6.700937,4
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 16:05:39,WINGS - BBQ,2.0,0,0,0.0,1.0,7.90,8.897042,4
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 16:59:35,TROPIC 4PLAY DBL,3.0,0,0,0.0,1.0,11.50,12.951391,4
362626040,2019-01-01 15:44:00,2019-01-01 17:16:01,01:32:01,2019-01-01 16:59:35,TROPIC 4PLAY DBL,3.0,0,0,0.0,1.0,11.50,12.951391,4


`After Work`

#### 9. Table 359970875

In [118]:
silvester[silvester.index == 359970875].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Kung Pao Chicken,2.0,0,0,0.0,1.0,16.99,19.197577,3
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Mexican Chicken,2.0,0,0,-0.0,-1.0,0.00,0.000000,3
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Classic Burger,2.0,0,0,0.0,1.0,12.99,14.677841,3
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378502,3
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Fount Pop,4.0,0,0,0.0,1.0,2.99,3.378502,3
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Kung Pao Chicken,2.0,0,0,0.0,1.0,16.99,19.197577,3
359970875,2018-12-26 13:37:28,2018-12-26 14:22:00,00:44:32,2018-12-26 13:38:19,Kung Pao Beef,2.0,0,0,-0.0,-1.0,0.00,0.000000,3


`Lunch`

#### 10. Table 363694039

In [119]:
silvester[silvester.index == 363694039].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:31:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508631,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:31:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508631,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:31:00,KIDBEV,4.0,0,0,0.0,1.0,0.00,0.000000,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:35:00,S/O.Bf.Gravy,2.0,0,0,0.0,1.0,1.49,1.683674,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:35:00,House,2.0,0,0,0.0,1.0,0.00,0.000000,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:35:00,K.PASTA CHS,2.0,0,0,0.0,1.0,7.81,8.825165,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:35:00,CHEESE.BURGER+,2.0,0,0,0.0,1.0,16.68,18.848111,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:35:00,KIDS.,2.0,0,0,0.0,1.0,0.00,0.000000,3
363694039,2019-01-04 17:31:00,2019-01-04 18:18:20,00:47:20,2019-01-04 17:35:00,BOSS.HOG.BURGER,2.0,0,0,0.0,1.0,17.97,20.305789,3


`Family Event`

#### 11. Table 361723648

In [120]:
silvester[silvester.index == 361723648].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:16:53,65cl Stella,1.0,44,2,0.651,1.0,10.75,12.147167,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:16:59,Fount Pop,4.0,0,0,0.000,1.0,2.99,3.378607,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:16:59,Fount Pop,4.0,0,0,0.000,1.0,2.99,3.378607,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:16:59,Fount Pop,4.0,0,0,0.000,1.0,2.99,3.378607,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:33:46,Caesar SM,2.0,0,0,0.000,1.0,6.49,7.333499,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:33:46,Caesar SM,2.0,0,0,0.000,1.0,6.49,7.333499,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:34:15,Tex Steak Sand,2.0,0,0,0.000,1.0,15.99,18.068204,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:34:15,1/2 Chk Fajita,2.0,0,0,0.000,1.0,17.99,20.328142,4
361723648,2018-12-30 16:16:46,2018-12-30 17:17:50,01:01:04,2018-12-30 16:34:15,Fish & Chips,2.0,0,0,0.000,1.0,13.99,15.808266,4


`Dinner`

#### 12. Table 360802830

In [121]:
silvester[silvester.index == 360802830].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:26:00,BANQUET.Pt,1.0,1120,2,0.473,1.0,8.43,9.525355,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:26:00,KIDPOP,4.0,0,0,0.000,1.0,0.00,0.000000,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:26:00,COORS.LIGHT.Pt,1.0,280,2,0.473,1.0,7.62,8.610107,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:34:00,DBL.PLAIN.BURG,2.0,0,0,0.000,1.0,15.29,17.276711,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:34:00,CAESAR.APPETIZER,2.0,0,0,0.000,1.0,6.35,7.175089,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:34:00,WINGS.15PC,2.0,0,0,0.000,1.0,23.65,26.722970,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:34:00,S/O.Bf.Gravy,2.0,0,0,0.000,1.0,1.49,1.683604,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:34:00,Add.Fries^,2.0,0,0,0.000,1.0,2.97,3.355908,3
360802830,2018-12-28 14:26:00,2018-12-28 15:57:34,01:31:34,2018-12-28 14:34:00,KIDS.,2.0,0,0,0.000,1.0,0.00,0.000000,3


`Family Event`

#### 13. Table 360778622

In [122]:
silvester[silvester.index == 360778622].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:39:00,V.STRAW.MINT.SODA,4.0,0,0,0.0,1.0,4.95,5.593017,3
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:39:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,3
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:42:00,TUNA.SALAD+,2.0,0,0,0.0,1.0,19.87,22.451161,3
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:42:00,Side Soup,2.0,0,0,0.0,1.0,0.00,0.000000,3
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:42:00,CHICKEN.CLUB,2.0,0,0,0.0,1.0,16.98,19.185743,3
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:42:00,WINGS.5PC,2.0,0,0,0.0,1.0,10.47,11.830078,3
360778622,2018-12-28 12:39:00,2018-12-28 13:20:25,00:41:25,2018-12-28 12:42:00,WINGS..,2.0,0,0,0.0,1.0,0.00,0.000000,3


`Lunch`

#### 14. Table 360543623

In [123]:
silvester[silvester.index == 360543623].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 15:02:19,1L Organic,1.0,350,2,1.0,1.0,11.00,12.430240,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 15:11:19,1L Organic,1.0,350,2,1.0,1.0,11.00,12.430240,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 15:19:42,8oz Steak Frites,2.0,0,0,0.0,1.0,20.99,23.719158,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 15:23:18,Crispy Calamari,2.0,0,0,0.0,1.0,13.99,15.809005,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 15:38:48,8oz Steak Frites,2.0,0,0,0.0,1.0,20.99,23.719158,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 15:46:02,1L Organic,1.0,350,2,1.0,1.0,11.00,12.430240,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 16:14:01,1L Organic,1.0,350,2,1.0,1.0,11.00,12.430240,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 16:21:02,OMELETTE TOPPING,2.0,0,0,0.0,1.0,0.99,1.118722,3
360543623,2018-12-27 15:02:08,2018-12-27 18:08:17,03:06:09,2018-12-27 16:21:02,SIDE Fries,2.0,0,0,0.0,1.0,3.99,4.508787,3


`Lunch`

#### 15. Table 363655255

In [124]:
silvester[silvester.index == 363655255].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
363655255,2019-01-04 14:35:12,2019-01-04 15:38:40,01:03:28,2019-01-04 14:35:34,The Fressor,2.0,0,0,0.0,1.0,19.98,22.576079,3
363655255,2019-01-04 14:35:12,2019-01-04 15:38:40,01:03:28,2019-01-04 14:35:34,Great Reuben,2.0,0,0,0.0,1.0,17.98,20.316212,3
363655255,2019-01-04 14:35:12,2019-01-04 15:38:40,01:03:28,2019-01-04 14:35:34,Great Reuben,2.0,0,0,0.0,1.0,14.99,16.937709,3


`Lunch`

#### 16. Table 360057302

In [125]:
silvester[silvester.index == 360057302].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:29:00,CALAMARI,2.0,0,0,0.0,1.0,14.35,16.215037,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:29:00,BRUSCHETTA.,2.0,0,0,0.0,1.0,13.31,15.039870,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,Salmon.Rice/Veg,2.0,0,0,0.0,1.0,0.00,0.000000,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,SMOKIN.SALMON,2.0,0,0,0.0,1.0,23.96,27.074026,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,BUBBLE.BOWL,3.0,0,0,0.0,1.0,16.99,19.198151,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,BUBBLE.BOWL,3.0,0,0,0.0,1.0,16.99,19.198151,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,V.SASSAFRAZ,4.0,0,0,0.0,1.0,4.95,5.593340,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,SMOKIN.SALMON,2.0,0,0,0.0,1.0,23.96,27.074026,4
360057302,2018-12-26 19:29:00,2018-12-26 20:27:59,00:58:59,2018-12-26 19:40:00,VEGGIE.BURGER,2.0,0,0,0.0,1.0,17.62,19.910031,4


`Dinner`

#### 17. Table 361881265

In [126]:
silvester[silvester.index == 361881265].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 21:43:00,PINK WRHSE,3.0,0,0,0.0,1.0,12.75,14.406957,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 22:28:00,SOUR KEY,3.0,0,0,0.0,1.0,6.75,7.627213,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 22:28:00,KEY-RONA,3.0,0,0,0.0,1.0,12.75,14.406957,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 22:28:00,WRHSE LEMONADE,3.0,0,0,0.0,1.0,12.75,14.406957,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 22:55:00,PERFECT STORM,3.0,0,0,0.0,1.0,7.25,8.192191,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 23:44:00,PINK LEMONADE,3.0,0,0,0.0,1.0,6.75,7.627213,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 23:44:00,SOUR KEY,3.0,0,0,0.0,1.0,6.75,7.627213,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 23:44:00,PINK LEMONADE,3.0,0,0,0.0,1.0,6.75,7.627213,3
361881265,2018-12-30 21:43:00,2018-12-30 23:53:36,02:10:36,2018-12-30 23:44:00,SOUR KEY,3.0,0,0,0.0,1.0,6.75,7.627213,3


`Drinking`

#### 18. Table 360336485

In [127]:
silvester[silvester.index == 360336485].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:38:00,JACKS.VEG.NACHOS,2.0,0,0,0.0,1.0,17.76,20.069434,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:38:00,WATER.,4.0,0,0,0.0,1.0,0.00,0.000000,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:38:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508842,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:38:00,POP.,4.0,0,0,0.0,1.0,3.99,4.508842,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:45:00,CHICKEN.FINGERS,2.0,0,0,0.0,1.0,15.99,18.069271,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:45:00,FINGERS.SAUCED,2.0,0,0,0.0,1.0,16.87,19.063702,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:45:00,FINGERS.SAUCED,2.0,0,0,0.0,1.0,16.87,19.063702,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:45:00,House,2.0,0,0,0.0,1.0,0.00,0.000000,3
360336485,2018-12-27 13:38:00,2018-12-27 14:26:03,00:48:03,2018-12-27 13:45:00,$Sub.Swt.Pot.Fry-,2.0,0,0,0.0,1.0,2.97,3.356206,3


`Social Gathering`

#### 19. Table 359420117

In [128]:
silvester[silvester.index == 359420117].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:31:00,PF-GARDEN.SALAD,2.0,0,0,0.0,1.0,0.0,0.0,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:31:00,PF-SEASON.SOUP,2.0,0,0,0.0,1.0,0.0,0.0,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:31:00,PF-SALMON.FLTBRD,2.0,0,0,0.0,1.0,0.0,0.0,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:31:00,PF-GARDEN.SALAD,2.0,0,0,0.0,1.0,0.0,0.0,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:35:00,PF-SOUTH.WEST.CHK,2.0,0,0,0.0,1.0,40.0,45.2,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:35:00,PF-SOUTH.WEST.CHK,2.0,0,0,0.0,1.0,40.0,45.2,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:35:00,PF-PAN.SALMON.,2.0,0,0,0.0,1.0,40.0,45.2,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 12:35:00,PF-CHIPOTLE.TOP,2.0,0,0,0.0,1.0,40.0,45.2,4
359420117,2018-12-24 12:31:00,2018-12-24 13:45:43,01:14:43,2018-12-24 13:16:00,PF-BROWNIE,2.0,0,0,0.0,1.0,0.0,0.0,4


`Lunch`

#### 20. Table 361326979

In [129]:
silvester[silvester.index == 361326979].sort_values(by=['order_item_time'])

,order_time,order_time_closed,dwell_time,order_item_time,title,category_id,beer_brand_id,beer_serving_type_id,beer_volume,item_qty,sales_before_tax,sales_inc_tax,guest_count
order_id,,,,,,,,,,,,,
361326979,2018-12-29 17:41:58,2018-12-29 18:25:14,00:43:16,2018-12-29 17:42:28,K- Chic Fajitas,2.0,0,0,0.0,1.0,9.99,11.287777,4
361326979,2018-12-29 17:41:58,2018-12-29 18:25:14,00:43:16,2018-12-29 17:42:28,Meatloaf,2.0,0,0,0.0,1.0,13.99,15.807408,4
361326979,2018-12-29 17:41:58,2018-12-29 18:25:14,00:43:16,2018-12-29 17:42:28,Classic Burger,2.0,0,0,0.0,1.0,12.99,14.677500,4
361326979,2018-12-29 17:41:58,2018-12-29 18:25:14,00:43:16,2018-12-29 17:42:28,Big Blue Burger,2.0,0,0,0.0,1.0,14.99,16.937315,4


`Family Event`